In [ ]:
###################################
#  These code are for style G from clusters 6-8.

#The idea is to write similar code for the other styles in these clusters and then combine them into one single script.

# Old Code: IGNORE: This is for a subset of projects-

## This is the new code which works for a subset

In [4]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate

def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            header = re.sub(r'\s+', ' ', header)
            header = re.sub(r'\(.*?\)', '', header)
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            header = header.strip()
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    return value

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()


def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.

    Args:
        title (str): The table title string.

    Returns:
        str: The extracted specific phrase if found, else the original title.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
        "ADNU",
        "LDNU",
        "RNU"
    ]

    for phrase in phrases:
        if re.search(rf"\b{re.escape(phrase)}\b", title, re.IGNORECASE):
            return phrase
    return title  # Fallback to the entire title if no specific phrase is found



def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type of upgrade",
        "upgrade",
        "description",
        "cost allocation factor"
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df


def search_gps_coordinates(text):
    """Search for GPS coordinates using multiple patterns."""
    print("Searching for GPS coordinates...")
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}")
        return gps_coords.groups()

    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}")
        return project_coords.groups()

    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
        longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
        print(f"Found directional GPS coordinates: {(latitude, longitude)}")
        return (latitude, longitude)

    print("GPS coordinates not found.")
    return (None, None)


 

def extract_table1(pdf_path):
    """
    Extracts the Point of Interconnection from Table 1 in the provided PDF.
    Implements a retry mechanism with different table extraction settings if initial attempts fail.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        str: Extracted Point of Interconnection value,
             "Value Missing" if label found but no value,
             or None if not found.
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...")
    point_of_interconnection = None

    # Define the regex pattern for 'Point of Interconnection' (case-insensitive)
    poi_pattern = re.compile(r"Point\s+of\s+Interconnection", re.IGNORECASE)

    # Define different table extraction settings to try
    table_settings_list = [
        {
            "horizontal_strategy": "text",
            "vertical_strategy": "lines",
            "snap_tolerance": 1,
        },
        {
            "horizontal_strategy": "lines",
            "vertical_strategy": "lines",
            "snap_tolerance": 2,  # Increased tolerance for retry
        }
    ]

    with pdfplumber.open(pdf_path) as pdf:
        # Identify all pages that contain "Table 1"
        table1_pages = []
        for i, page in enumerate(pdf.pages):
            text = page.extract_text() or ""
            if re.search(r"Table\s*1\b", text, re.IGNORECASE):
                table1_pages.append(i)

        if not table1_pages:
            print("No Table 1 found in the PDF.")
            return None  # Return None if no Table 1 found

        first_page = table1_pages[0]
        last_page = table1_pages[-1]
        scrape_start = first_page
        scrape_end = last_page + 1  # Plus one to include the next page if needed

        print(f"Table 1 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}")

        # Flag to indicate if extraction was successful
        extraction_successful = False

        # Iterate through the specified page range
        for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
            page = pdf.pages[page_number]
            print(f"\nScraping tables on page {page_number + 1} for Table 1...")

            for attempt, table_settings in enumerate(table_settings_list, start=1):
                print(f"\nAttempt {attempt} with table settings: {table_settings}")
                tables = page.find_tables(table_settings=table_settings)
                print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.")

                for table_index, table in enumerate(tables, start=1):
                    tab = table.extract()
                    if not tab:
                        print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.")
                        continue  # Skip empty tables

                    print(f"\n--- Table {table_index} on Page {page_number + 1} ---")
                    for row_num, row in enumerate(tab, start=1):
                        print(f"Row {row_num}: {row}")

                    # Iterate through each row in the table
                    for row_index, row in enumerate(tab, start=1):
                        # Iterate through each cell in the row
                        for cell_index, cell in enumerate(row, start=1):
                            if cell and poi_pattern.search(cell):
                                # Assuming the next column contains the value
                                poi_col_index = cell_index  # 1-based index
                                adjacent_col_index = poi_col_index + 1  # Next column

                                if adjacent_col_index <= len(row):
                                    poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                    if poi_value:  # Check if the value is not empty
                                        point_of_interconnection = poi_value
                                        print(f"\nFound Point of Interconnection: '{point_of_interconnection}' "
                                              f"(Page {page_number + 1}, Table {table_index}, Row {row_index})")
                                        extraction_successful = True
                                        break  # Exit the cell loop
                                    else:
                                        print(f"\nPoint of Interconnection label found but adjacent value is empty "
                                              f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).")
                                        # Proceed to scan surrounding rows for the value
                                        poi_value_parts = []

                                        # Define the range to scan: two rows above and two rows below
                                        # Convert to 0-based index
                                        current_row_idx = row_index - 1
                                        start_scan = max(0, current_row_idx - 2)
                                        end_scan = min(len(tab), current_row_idx + 3)  # Exclusive

                                        print(f"Scanning rows {start_scan + 1} to {end_scan} for POI value parts.")

                                        for scan_row_index in range(start_scan, end_scan):
                                            # Skip the current row where the label was found
                                            if scan_row_index == current_row_idx:
                                                continue

                                            scan_row = tab[scan_row_index]
                                            # Ensure the adjacent column exists in the scan row
                                            if adjacent_col_index - 1 < len(scan_row):
                                                scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                if scan_cell and not poi_pattern.search(scan_cell):
                                                    poi_value_parts.append(scan_cell)
                                                    print(f"Found POI part in row {scan_row_index + 1}: '{scan_cell}'")
                                                elif poi_pattern.search(scan_cell):
                                                    # If another POI label is found, skip it
                                                    print(f"Encountered another POI label in row {scan_row_index + 1}. Skipping this row.")
                                                    continue

                                        if poi_value_parts:
                                            # Concatenate the parts to form the complete POI value
                                            point_of_interconnection = " ".join(poi_value_parts)
                                            print(f"\nConcatenated Point of Interconnection: '{point_of_interconnection}' "
                                                  f"(Page {page_number + 1}, Table {table_index})")
                                            extraction_successful = True
                                            break  # Exit the cell loop
                                        else:
                                            print(f"\nNo POI value found in the surrounding rows "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).")
                                            # Do not return immediately; proceed to retry
                                else:
                                    print(f"\nPoint of Interconnection label found but no adjacent column "
                                          f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).")
                                    # Do not return immediately; proceed to retry
                        if extraction_successful:
                            break  # Exit the row loop
                    if extraction_successful:
                        break  # Exit the table loop
                if extraction_successful:
                    break  # Exit the attempt loop
            if extraction_successful:
                break  # Exit the page loop

    if not extraction_successful:
        # After all attempts, determine the appropriate return value
        if point_of_interconnection is not None and point_of_interconnection != "":
            # Label was found but no value
            print("Point of Interconnection label found but no adjacent value.")
            return "Value Missing"
        else:
            # Label not found
            print("Point of Interconnection not found in Table 1.")
            return None

    return point_of_interconnection





def extract_base_data(pdf_path, project_id):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...")
    with open(pdf_path, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text

    text = clean_string_cell(text)

    queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
    queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found
    print(f"Extracted Queue ID: {queue_id}")

    cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
    cluster_number = cluster_number.group(1) if cluster_number else None
    print(f"Extracted Cluster Number: {cluster_number}")

    deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
    deliverability_status = deliverability_status.group(1) if deliverability_status else None
    print(f"Extracted Deliverability Status: {deliverability_status}")

    # Extract Capacity
    capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
    if capacity:
        capacity = int(capacity.group(1))
    else:
        capacity2 = re.search(r"(\d+)\s*mw", text)
        capacity = int(capacity2.group(1)) if capacity2 else None
    print(f"Extracted Capacity: {capacity}")

    # Removed Point of Interconnection extraction as per instruction
    point_of_interconnection = extract_table1(pdf_path)
    #point_of_interconnection = None
    latitude, longitude = search_gps_coordinates(text)

    # Initialize base data dictionary
    base_data = {
        "q_id": [queue_id],
        "cluster": [cluster_number],
        "req_deliverability": [deliverability_status],
        "latitude": [latitude],
        "longitude": [longitude],
        "capacity": [capacity],
        "point_of_interconnection": [point_of_interconnection]
    }

    print("Base data extracted:", base_data)
    return pd.DataFrame(base_data)





def extract_table7(pdf_path):
    """
    Extracts Table 7 data from the provided PDF.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        pd.DataFrame: Extracted Table 7 data.
    """
    print(f"\nProcessing {pdf_path} for Table 7 extraction...")
    extracted_tables = []
    specific_phrase = None  # To store the specific phrase from the table title

    with pdfplumber.open(pdf_path) as pdf:
        # Identify all pages that contain "Table 7" or "Table 7-"
        table7_pages = []
        for i, page in enumerate(pdf.pages):
            text = page.extract_text() or ""
            if re.search(r"Table\s*7", text, re.IGNORECASE): #[-\s]
                table7_pages.append(i)

        if not table7_pages:
            print("No Table 7 found in the PDF.")
            return pd.DataFrame()  # Return empty DataFrame if no Table 7 found

        first_page = table7_pages[0]
        last_page = table7_pages[-1]
        scrape_start = first_page
        scrape_end = last_page + 1  # Plus one to include the next page

        print(f"Table 7 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}")

        # Iterate through the specified page range
        for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
            page = pdf.pages[page_number]
            print(f"\nScraping tables on page {page_number + 1}...")
            tables = page.find_tables(table_settings={
                "horizontal_strategy": "lines",
                "vertical_strategy": "lines",
            })

            for table_index, table in enumerate(tables):
                tab = table.extract()
                if not tab:
                    print(f"Table {table_index + 1} on page {page_number + 1} is empty. Skipping.")
                    continue  # Skip empty tables

                # Get the bounding box of the current table
                table_bbox = table.bbox  # (x0, y0, x1, y1)

                # Define a bounding box from the top of the page to the top of the table
                # Coordinates: (x0, y0, x1, y1)
                title_bbox = (0, 0, page.width, table_bbox[1])

                # Extract text within the title bounding box
                title_text = page.within_bbox(title_bbox).extract_text() or ""

                # Initialize table_title as None
                table_title = None

                if title_text:
                    # Split the title text into lines and reverse to start checking from the closest line to the table
                    title_lines = title_text.split('\n')[::-1]

                    for line in title_lines:
                        line = line.strip()
                        # Check if the line matches the table title pattern (e.g., "Table 7-1: PTO Upgrade")
                        match = re.match(r"Table\s*7[-.]\d+[:\-\s]*(.*)", line, re.IGNORECASE)
                        if match:
                            table_title = match.group(1).strip()
                            break  # Stop after finding the closest table title

                if table_title:
                    # New Table 7 detected
                    specific_phrase = extract_specific_phrase(table_title)  # Extracted phrase from the title
                    print(f"New Table 7 detected: '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}")

                    # Extract headers and data rows
                    headers = clean_column_headers(tab[0])
                    data_rows = tab[1:]

                    # Create DataFrame for the new table
                    try:
                        df_new = pd.DataFrame(data_rows, columns=headers)
                    except ValueError as ve:
                        print(f"Error creating DataFrame for new table on page {page_number + 1}, table {table_index + 1}: {ve}")
                        continue  # Skip this table due to error

                    # Special Handling for "Area Delivery Network Upgrade" Tables
                    if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                        print("Detected 'Area Delivery Network Upgrade' table.")
                        if "adnu" in df_new.columns:
                            # Rename 'adnu' to 'upgrade'
                            if "upgrade" in df_new.columns:
                                # Avoid duplicate 'upgrade' columns
                                df_new.drop(columns=['adnu'], inplace=True)
                                print("Dropped 'ADNU' column to avoid duplicate 'upgrade' columns.")
                            else:
                                df_new.rename(columns={'adnu': 'upgrade'}, inplace=True)
                                print("Renamed 'ADNU' to 'upgrade'.")
                        if "type of upgrade" not in df_new.columns:
                            df_new["type of upgrade"] = specific_phrase
                            print("Added 'type of upgrade' column with specific phrase.")
                    else:
                        # General Handling for other tables
                        if "type of upgrade" in df_new.columns:
                            first_row = df_new.iloc[0]
                            if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                print(f"Replacing 'None' in 'Type of Upgrade' for the first data row on page {page_number + 1}, table {table_index + 1}")
                                df_new.at[0, "type of upgrade"] = specific_phrase
                        else:
                            # If "Type of Upgrade" column does not exist, add it
                            df_new["type of upgrade"] = specific_phrase
                            print(f"'Type of Upgrade' column added with value '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}")

                    # Ensure no duplicate columns
                    if df_new.columns.duplicated().any():
                        print(f"Duplicate columns detected in new table on page {page_number + 1}, table {table_index + 1}. Dropping duplicates.")
                        df_new = df_new.loc[:, ~df_new.columns.duplicated()]

                    # Append the new table to the list
                    extracted_tables.append(df_new)
                else:
                    # Continuation Table detected
                    if specific_phrase is None:
                        print(f"No previous Table 7 title found for continuation on page {page_number + 1}, table {table_index + 1}. Skipping.")
                        continue  # Skip if no previous title to refer to

                    print(f"Continuation Table detected on page {page_number + 1}, table {table_index + 1}")

                    # Treat all rows as data without headers
                    data_rows = tab

                    # Check if the number of columns matches
                    expected_columns = len(extracted_tables[-1].columns) if extracted_tables else None
                    if expected_columns is None:
                        print(f"No existing table to continue with on page {page_number + 1}, table {table_index + 1}. Skipping.")
                        continue  # No table to continue with

                    # Define expected columns based on the last extracted table
                    expected_headers = extracted_tables[-1].columns.tolist()

                    # Define the list of column headers to check in continuation tables
                    header_keywords = ["type of upgrade", "upgrade", "adnu"]

                    # Check if the first row contains any header keywords
                    first_continuation_row = data_rows[0] if len(data_rows) > 0 else []
                    is_header_row = any(
                        re.search(rf"\b{kw}\b", str(cell), re.IGNORECASE) for kw in header_keywords for cell in first_continuation_row
                    )

                    if is_header_row:
                        print(f"Detected header row in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping the header row.")
                        data_rows = data_rows[1:]  # Drop the header row

                    # Special Handling for "Area Delivery Network Upgrade" Tables in Continuation
                    if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                        print("Handling continuation for 'Area Delivery Network Upgrade' table.")
                        if "adnu" in data_rows[0]:
                            # Rename 'ADNU' to 'upgrade' in continuation table.
                            print("Renaming 'ADNU' to 'upgrade' in continuation table.")
                            data_rows = [ ["upgrade"] + row[1:] for row in data_rows ]
                        if "upgrade" not in expected_headers:
                            # Insert 'upgrade' column at the beginning
                            print("Inserting 'upgrade' column with specific phrase in continuation table.")
                            data_rows = [ [specific_phrase] + row for row in data_rows ]

                    # Handle missing or extra columns
                    if len(data_rows[0]) < expected_columns:
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            # For ADNU tables, assume missing "upgrade" column
                            print(f"Detected missing 'upgrade' column in continuation table on page {page_number + 1}, table {table_index + 1}. Inserting 'upgrade' column.")
                            data_rows = [[specific_phrase] + row for row in data_rows]
                        else:
                            # For other tables, assume missing "Type of Upgrade" column
                            print(f"Detected missing 'Type of Upgrade' column in continuation table on page {page_number + 1}, table {table_index + 1}. Inserting 'Type of Upgrade' column.")
                            data_rows = [["Type of Upgrade"] + row for row in data_rows]
                    elif len(data_rows[0]) > expected_columns:
                        # Extra columns detected; adjust accordingly
                        print(f"Detected extra columns in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping extra columns.")
                        data_rows = [row[:expected_columns] for row in data_rows]

                    # Create DataFrame for the continuation table
                    try:
                        df_continuation = pd.DataFrame(data_rows, columns=expected_headers)
                    except ValueError as ve:
                        print(f"Error creating DataFrame for continuation table on page {page_number + 1}, table {table_index + 1}: {ve}")
                        continue  # Skip this table due to error

                    # Special Handling for "Area Delivery Network Upgrade" Tables in Continuation
                    if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                        if "upgrade" in df_continuation.columns:
                            first_row = df_continuation.iloc[0]
                            if pd.isna(first_row["upgrade"]) or first_row["upgrade"] == "":
                                print(f"Replacing 'None' in 'upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}")
                                df_continuation.at[0, "upgrade"] = specific_phrase
                        else:
                            # If "upgrade" column does not exist, add it
                            df_continuation["upgrade"] = specific_phrase
                            print(f"'upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}")
                    else:
                        # General Handling for other tables
                        if "type of upgrade" in df_continuation.columns:
                            first_row = df_continuation.iloc[0]
                            if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                print(f"Replacing 'None' in 'Type of Upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}")
                                df_continuation.at[0, "type of upgrade"] = specific_phrase
                        else:
                            # If "Type of Upgrade" column does not exist, add it
                            df_continuation["type of upgrade"] = specific_phrase
                            print(f"'Type of Upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}")

                    # Ensure no duplicate columns
                    if df_continuation.columns.duplicated().any():
                        print(f"Duplicate columns detected in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping duplicates.")
                        df_continuation = df_continuation.loc[:, ~df_continuation.columns.duplicated()]

                    # Merge with the last extracted table
                    extracted_tables[-1] = pd.concat([extracted_tables[-1], df_continuation], ignore_index=True, sort=False)

    # After scraping all tables, concatenate them
    if extracted_tables:
        # To prevent reindexing errors, ensure all DataFrames have unique and consistent columns
        # Collect all unique column names
        all_columns = set()
        for df in extracted_tables:
            all_columns.update(df.columns.tolist())
        #all_columns = sorted(all_columns)  # Optional: sort columns for consistency

        standardized_tables = []
        for df in extracted_tables:
            # Reindex to have all columns, filling missing ones with NaN
            standardized_df = df.reindex(columns=all_columns)
            standardized_tables.append(standardized_df)

        print("\nConcatenating all extracted Table 7 data...")
        try:
            table7_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
        except Exception as e:
            print(f"Error concatenating tables: {e}")
            table7_data = pd.DataFrame()
    else:
        print("No Table 7 data extracted.")
        table7_data = pd.DataFrame()

    return table7_data

# Example Usage
def main():
    pdfs = [    
       #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/947/02_phase_1_study/Q947 GV Solar 3_C6Ph I_Appendix A-Individual Study Report_Final.pdf",
       "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1010/02_phase_1_study/C7PhI Appendix A - Q1010 Dyer Summit Wind Repower.pdf",
       "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1021/02_phase_1_study/Q1021_StocktonBiomass_PHI_Report-Addendum1.pdf",
        #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf",
        #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1035/02_phase_1_study/C7PhI Appendix A - Q1035 Magellan Solar Final.pdf",
        #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1098/02_phase_1_study/C8 Ph I Appendix A - Q1098 Birds Landing Wind.pdf",
        #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1130/02_phase_1_study/Q1130 Moon Prism 2_Appendix A Report_C8PhI.pdf",
        #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1133/02_phase_1_study/Q1133 Oveja Solar Farm_Appendix A Report_C8PhI.pdf",
        #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1158/02_phase_1_study/Q1158 Slate_Appendix A Report_C8PhI.pdf",
        #"/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1170/02_phase_1_study/QC8PhI_Q1170_Gateway Energy Storage_Appendix A_1-15-2016.pdf",
    ]

    # Initialize an empty DataFrame to store all results
    core = pd.DataFrame()

    for pdf_path in pdfs:
        print(f"\nProcessing PDF: {pdf_path}")
        # Extract base data
        project_id = os.path.basename(pdf_path).split('.')[0]  # Modify as appropriate
        base_data = extract_base_data(pdf_path, project_id)

        # Extract Table 7 data
        table7_data = extract_table7(pdf_path)

        if table7_data.empty:
            print(f"No Table 7 data found in {pdf_path}. Using base data only.")
            merged_df = base_data
        else:
            # Merge base data with Table 7 data
            overlapping_columns = base_data.columns.intersection(table7_data.columns)
            table7_data = table7_data.drop(columns=overlapping_columns, errors='ignore')
            base_data_repeated = pd.concat([base_data] * len(table7_data), ignore_index=True)
            try:
                merged_df = pd.concat([base_data_repeated, table7_data], axis=1, sort=False)
                print(f"Merged base data with Table 7 data for {pdf_path}")
            except Exception as e:
                print(f"Error merging base data with Table 7 data for {pdf_path}: {e}")
                merged_df = base_data  # Fallback to base data only

        # Append to core
        try:
            core = pd.concat([core, merged_df], ignore_index=True, sort=False)
        except Exception as e:
            print(f"Error appending data from {pdf_path} to core DataFrame: {e}")

    # Clean up the entire DataFrame
    core = core.applymap(clean_string_cell)

    # Reorder columns as specified
    core = reorder_columns(core)
    print("\nColumns reordered as per specification.")

    # Display the final combined DataFrame
    print("\nFinal combined DataFrame:")
    print(tabulate(core, headers='keys', tablefmt='fancy_grid', showindex=False))

    # Save to CSV if needed
    # core.to_csv('final_data.csv', index=False)

if __name__ == "__main__":
    main()



Processing PDF: /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/1010/02_phase_1_study/C7PhI Appendix A - Q1010 Dyer Summit Wind Repower.pdf
Extracting base data from PDF...
Extracted Queue ID: 1010
Extracted Cluster Number: 7
Extracted Deliverability Status: Full
Extracted Capacity: None

Processing /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/1010/02_phase_1_study/C7PhI Appendix A - Q1010 Dyer Summit Wind Repower.pdf for Table 1 extraction...
Table 1 starts on page 4 and ends on page 5

Scraping tables on page 4 for Table 1...

Attempt 1 with table settings: {'horizontal_strategy': 'text', 'vertical_strategy': 'lines', 'snap_tolerance': 1}
Found 1 table(s) on page 4 with current settings.

--- Table 1 on Page 4 ---
Row 1: ['Project Parameters', 'Project Specific Data']
Row 2: ['', '']
Row 3: ['', 'Tracy, Alameda County, CA']
Row 4: ['', '']
Row 5: ['Project Location', '']
Row 6: ['', '']
Row 7: ['', 'GPS Coordinates: 37.76122°, -121.690893°']
Row 

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_75037/2247125055.py:648: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  core = core.applymap(clean_string_cell)


In [75]:
import pdfplumber

with pdfplumber.open("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/947/02_phase_1_study/Q947 GV Solar 3_C6Ph I_Appendix A-Individual Study Report_Final.pdf") as pdf:
    page = pdf.pages[4]
    table = page.extract_table()
    print(table)


[['Project Parameters Pro', 'ject Specific Data'], ['Alpa\nProject Location\nGPS', 'ugh, Tulare County, CA\nCoordinates: 35° 49.285 N, 119° 27.317 W'], ['Participating TO’s Planning Area Kern', ''], ['240\nNumber and Type of Generators\nInve', 'x 0.5 MW PV Advanced Energy Solaron\nrter Units'], ['Interconnection Voltage 115', 'kV'], ['Maximum Generator Output at\n120\ngenerator Terminal', 'MW'], ['Generator Auxiliary Load 0.5', 'MW'], ['Maximum Net Output 120', 'MW'], ['Power Factor Range 0.95', 'leading to 0.95 lagging'], ['thre\nStep-up Transformer(s)\nwith', 'e-phase 115/34.5, rated for 75/100/125 MVA\nZ=8.5% at 75 MVA'], ['Collector System Equivalent\nR1\nImpedance P', '=0.0083 X1 =0.0067 B1 =0.0240\nU PU PU'], ['Point of Interconnection Oliv', 'e 115kV Substation'], ['Interconnection Customer Requested\nDec\nCommercial Operation Date', 'ember 1, 2017']]


# All Projects in the cluster code: this code fixed the issues with the above code

# Old Codes Please IGNORE

In [ ]:
# Note some projects require some manual attention like 1098, 1099, 1102,1133

In [ ]:
#lets skip these projects for now 1133,1225,1227,1229,1234,1236,1237,1249,1252,1256,

# Final Code for Cluster 7 Style G.

In [11]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
import traceback

# Define paths and project range
BASE_DIRECTORY = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
OUTPUT_CSV_PATH_ORIGINAL = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/rawdata_cluster7_style_G_originals.csv"
OUTPUT_CSV_PATH_ADDENDUM = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/rawdata_cluster7_style_G_addendums.csv"
LOG_FILE_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/scraping_cluster7_style_G_log.txt"
PROJECT_RANGE = range(1007, 1089)  # Example range for q_ids in Clusters 6, 7, 8

# Initialize DataFrames
core_originals = pd.DataFrame()
core_addendums = pd.DataFrame()

# Initialize tracking variables
scraped_projects = set()
skipped_projects = set()
missing_projects = set()
scraped_pdfs = []
skipped_pdfs = []
addendum_pdfs = []
original_pdfs = []
total_pdfs_accessed = 0
total_pdfs_scraped = 0
total_pdfs_skipped = 0

def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            header = re.sub(r'\s+', ' ', header)
            header = re.sub(r'\(.*?\)', '', header)
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            header = header.strip()
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    return value

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.

    Args:
        title (str): The table title string.

    Returns:
        str: The extracted specific phrase if found, else the original title.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
        "ADNU",
        "LDNU",
        "RNU"
    ]

    for phrase in phrases:
        if re.search(rf"\b{re.escape(phrase)}\b", title, re.IGNORECASE):
            return phrase
    return title  # Fallback to the entire title if no specific phrase is found

def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type of upgrade",
        "upgrade",
        "description",
        "cost allocation factor"
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df

def search_gps_coordinates(text, log_file):
    """Search for GPS coordinates using multiple patterns."""
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}", file=log_file)
        return gps_coords.groups()

    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}", file=log_file)
        return project_coords.groups()

    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
        longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
        print(f"Found directional GPS coordinates: {(latitude, longitude)}", file=log_file)
        return (latitude, longitude)

    print("GPS coordinates not found.", file=log_file)
    return (None, None)

def extract_table1(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table 1 in the provided PDF.
    Implements a retry mechanism with different table extraction settings if initial attempts fail.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.

    Returns:
        str: Extracted Point of Interconnection value,
             "Value Missing" if label found but no value,
             or None if not found.
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...", file=log_file)
    point_of_interconnection = None

    # Define the regex pattern for 'Point of Interconnection' (case-insensitive)
    poi_pattern = re.compile(r"Point\s+of\s+Interconnection", re.IGNORECASE)

    # Define different table extraction settings to try
    table_settings_list = [
        {
            "horizontal_strategy": "text",
            "vertical_strategy": "lines",
            "snap_tolerance": 1,
        },
        {
            "horizontal_strategy": "lines",
            "vertical_strategy": "lines",
            "snap_tolerance": 2,  # Increased tolerance for retry
        }
    ]

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify all pages that contain "Table 1"
            table1_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*1\b", text, re.IGNORECASE):
                    table1_pages.append(i)

            if not table1_pages:
                print("No Table 1 found in the PDF.", file=log_file)
                return None  # Return None if no Table 1 found

            first_page = table1_pages[0]
            last_page = table1_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 1  # Plus one to include the next page if needed

            print(f"Table 1 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}", file=log_file)

            # Flag to indicate if extraction was successful
            extraction_successful = False

            # Iterate through the specified page range
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1} for Table 1...", file=log_file)

                for attempt, table_settings in enumerate(table_settings_list, start=1):
                    print(f"\nAttempt {attempt} with table settings: {table_settings}", file=log_file)
                    tables = page.find_tables(table_settings=table_settings)
                    print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.", file=log_file)

                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.", file=log_file)
                            continue  # Skip empty tables

                        print(f"\n--- Table {table_index} on Page {page_number + 1} ---", file=log_file)
                        for row_num, row in enumerate(tab, start=1):
                            print(f"Row {row_num}: {row}", file=log_file)

                        # Iterate through each row in the table
                        for row_index, row in enumerate(tab, start=1):
                            # Iterate through each cell in the row
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    # Assuming the next column contains the value
                                    poi_col_index = cell_index  # 1-based index
                                    adjacent_col_index = poi_col_index + 1  # Next column

                                    if adjacent_col_index <= len(row):
                                        poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                        if poi_value:  # Check if the value is not empty
                                            point_of_interconnection = poi_value
                                            print(f"\nFound Point of Interconnection: '{point_of_interconnection}' "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                            extraction_successful = True
                                            break  # Exit the cell loop
                                        else:
                                            print(f"\nPoint of Interconnection label found but adjacent value is empty "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                            # Proceed to scan surrounding rows for the value
                                            poi_value_parts = []

                                            # Define the range to scan: two rows above and two rows below
                                            # Convert to 0-based index
                                            current_row_idx = row_index - 1
                                            start_scan = max(0, current_row_idx - 2)
                                            end_scan = min(len(tab), current_row_idx + 3)  # Exclusive

                                            print(f"Scanning rows {start_scan + 1} to {end_scan} for POI value parts.", file=log_file)

                                            for scan_row_index in range(start_scan, end_scan):
                                                # Skip the current row where the label was found
                                                if scan_row_index == current_row_idx:
                                                    continue

                                                scan_row = tab[scan_row_index]
                                                # Ensure the adjacent column exists in the scan row
                                                if adjacent_col_index - 1 < len(scan_row):
                                                    scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                    if scan_cell and not poi_pattern.search(scan_cell):
                                                        poi_value_parts.append(scan_cell)
                                                        print(f"Found POI part in row {scan_row_index + 1}: '{scan_cell}'", file=log_file)
                                                    elif poi_pattern.search(scan_cell):
                                                        # If another POI label is found, skip it
                                                        print(f"Encountered another POI label in row {scan_row_index + 1}. Skipping this row.", file=log_file)
                                                        continue

                                            if poi_value_parts:
                                                # Concatenate the parts to form the complete POI value
                                                point_of_interconnection = " ".join(poi_value_parts)
                                                print(f"\nConcatenated Point of Interconnection: '{point_of_interconnection}' "
                                                      f"(Page {page_number + 1}, Table {table_index})", file=log_file)
                                                extraction_successful = True
                                                break  # Exit the cell loop
                                            else:
                                                print(f"\nNo POI value found in the surrounding rows "
                                                      f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                                # Do not return immediately; proceed to retry
                                    else:
                                        print(f"\nPoint of Interconnection label found but no adjacent column "
                                              f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                        # Do not return immediately; proceed to retry
                            if extraction_successful:
                                break  # Exit the row loop
                        if extraction_successful:
                            break  # Exit the table loop
                    if extraction_successful:
                        break  # Exit the attempt loop
                if extraction_successful:
                    break  # Exit the page loop

    except Exception as e:
        print(f"Error processing Table 1 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None

    if not extraction_successful:
        # After all attempts, determine the appropriate return value
        if point_of_interconnection is not None and point_of_interconnection != "":
            # Label was found but no value
            print("Point of Interconnection label found but no adjacent value.", file=log_file)
            return "Value Missing"
        else:
            # Label not found
            print("Point of Interconnection not found in Table 1.", file=log_file)
            return None

    return point_of_interconnection

def extract_base_data(pdf_path, project_id, log_file):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...", file=log_file)
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        text = clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found
        print(f"Extracted Queue ID: {queue_id}", file=log_file)

        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = cluster_number.group(1) if cluster_number else None
        print(f"Extracted Cluster Number: {cluster_number}", file=log_file)

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        print(f"Extracted Deliverability Status: {deliverability_status}", file=log_file)

        # Extract Capacity
        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        print(f"Extracted Capacity: {capacity}", file=log_file)

        # Extract Point of Interconnection
        point_of_interconnection = extract_table1(pdf_path, log_file)

        latitude, longitude = search_gps_coordinates(text, log_file)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        print("Base data extracted:", file=log_file)
        print(base_data, file=log_file)
        return pd.DataFrame(base_data)

    except Exception as e:
        print(f"Error extracting base data from {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()  # Return empty DataFrame on error

def extract_table7(pdf_path, log_file, is_addendum=False):
    """
    Extracts Table 7 data from the provided PDF.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.
        is_addendum (bool): Whether the PDF is an addendum.

    Returns:
        pd.DataFrame: Extracted Table 7 data.
    """
    print(f"\nProcessing {pdf_path} for Table 7 extraction...", file=log_file)
    extracted_tables = []
    specific_phrase = None  # To store the specific phrase from the table title

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify all pages that contain "Table 7" or variations like "Modification of Table 7"
            table7_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                # Use re.search to find "Table 7" anywhere in the text, allowing for prefixes
                if re.search(r"(Modification\s+of\s+)?Table\s*7[-.]?\d*", text, re.IGNORECASE):
                    table7_pages.append(i)

            if not table7_pages:
                print("No Table 7 found in the PDF.", file=log_file)
                return pd.DataFrame()  # Return empty DataFrame if no Table 7 found

            first_page = table7_pages[0]
            last_page = table7_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 1  # Plus one to include the next page

            print(f"Table 7 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}", file=log_file)

            # Iterate through the specified page range
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1}...", file=log_file)
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })

                for table_index, table in enumerate(tables):
                    tab = table.extract()
                    if not tab:
                        print(f"Table {table_index + 1} on page {page_number + 1} is empty. Skipping.", file=log_file)
                        continue  # Skip empty tables

                    # Get the bounding box of the current table
                    table_bbox = table.bbox  # (x0, y0, x1, y1)

                    # Define a bounding box from the top of the page to the top of the table
                    # Coordinates: (x0, y0, x1, y1)
                    title_bbox = (0, 0, page.width, table_bbox[1])

                    # Extract text within the title bounding box
                    title_text = page.within_bbox(title_bbox).extract_text() or ""

                    # Initialize table_title as None
                    table_title = None

                    if title_text:
                        # Split the title text into lines and reverse to start checking from the closest line to the table
                        title_lines = title_text.split('\n')[::-1]

                        for line in title_lines:
                            line = line.strip()
                            # Use re.search instead of re.match to find "Table 7" anywhere in the line
                            match = re.search(r"(Modification\s+of\s+)?Table\s*7[-.]?\d*[:\-\s]*(.*)", line, re.IGNORECASE)
                            if match:
                                # Capture the specific phrase after "Table 7" and possible number
                                table_title = match.group(2).strip()
                                break  # Stop after finding the closest table title

                    if table_title:
                        # New Table 7 detected
                        specific_phrase = extract_specific_phrase(table_title)  # Extracted phrase from the title
                        print(f"New Table 7 detected: '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        # Extract headers and data rows
                        headers = clean_column_headers(tab[0])
                        data_rows = tab[1:]

                        # Create DataFrame for the new table
                        try:
                            df_new = pd.DataFrame(data_rows, columns=headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for new table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue  # Skip this table due to error

                        # Special Handling for "Area Delivery Network Upgrade" Tables
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            print("Detected 'Area Delivery Network Upgrade' table.", file=log_file)
                            if "adnu" in df_new.columns:
                                # Rename 'adnu' to 'upgrade'
                                if "upgrade" in df_new.columns:
                                    # Avoid duplicate 'upgrade' columns
                                    df_new.drop(columns=['adnu'], inplace=True)
                                    print("Dropped 'ADNU' column to avoid duplicate 'upgrade' columns.", file=log_file)
                                else:
                                    df_new.rename(columns={'adnu': 'upgrade'}, inplace=True)
                                    print("Renamed 'ADNU' to 'upgrade'.", file=log_file)
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' column with specific phrase.", file=log_file)
                        else:
                            # General Handling for other tables
                            if "type of upgrade" in df_new.columns:
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing 'None' in 'Type of Upgrade' for the first data row on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                            else:
                                # If "Type of Upgrade" column does not exist, add it
                                df_new["type of upgrade"] = specific_phrase
                                print(f"'Type of Upgrade' column added with value '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        # Ensure no duplicate columns
                        if df_new.columns.duplicated().any():
                            print(f"Duplicate columns detected in new table on page {page_number + 1}, table {table_index + 1}. Dropping duplicates.", file=log_file)
                            df_new = df_new.loc[:, ~df_new.columns.duplicated()]

                        # Append the new table to the list
                        extracted_tables.append(df_new)
                    else:
                        # Continuation Table detected
                        if specific_phrase is None:
                            print(f"No previous Table 7 title found for continuation on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue  # Skip if no previous title to refer to

                        print(f"Continuation Table detected on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        # Treat all rows as data without headers
                        data_rows = tab

                        # Check if the number of columns matches
                        expected_columns = len(extracted_tables[-1].columns) if extracted_tables else None
                        if expected_columns is None:
                            print(f"No existing table to continue with on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue  # No table to continue with

                        # Define expected columns based on the last extracted table
                        expected_headers = extracted_tables[-1].columns.tolist()

                        # Define the list of column headers to check in continuation tables
                        header_keywords = ["type of upgrade", "upgrade", "adnu"]

                        # Check if the first row contains any header keywords
                        first_continuation_row = data_rows[0] if len(data_rows) > 0 else []
                        is_header_row = any(
                            re.search(rf"\b{kw}\b", str(cell), re.IGNORECASE) for kw in header_keywords for cell in first_continuation_row
                        )

                        if is_header_row:
                            print(f"Detected header row in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping the header row.", file=log_file)
                            data_rows = data_rows[1:]  # Drop the header row

                        # Special Handling for "Area Delivery Network Upgrade" Tables in Continuation
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            print("Handling continuation for 'Area Delivery Network Upgrade' table.", file=log_file)
                            if "adnu" in data_rows[0]:
                                # Rename 'ADNU' to 'upgrade' in continuation table.
                                print("Renaming 'ADNU' to 'upgrade' in continuation table.", file=log_file)
                                data_rows = [["upgrade"] + row[1:] for row in data_rows]
                            if "upgrade" not in expected_headers:
                                # Insert 'upgrade' column at the beginning
                                print("Inserting 'upgrade' column with specific phrase in continuation table.", file=log_file)
                                data_rows = [[specific_phrase] + row for row in data_rows]
                        else:
                            # General Handling for other tables
                            if "type of upgrade" in expected_headers:
                                # Ensure the first data row has the specific phrase
                                print("Ensuring 'type of upgrade' column has the specific phrase in continuation table.", file=log_file)
                            if "type of upgrade" not in expected_headers:
                                # Insert 'Type of Upgrade' column with specific phrase
                                print("Inserting 'Type of Upgrade' column with specific phrase in continuation table.", file=log_file)
                                data_rows = [[specific_phrase] + row for row in data_rows]

                        # Handle missing or extra columns
                        if len(data_rows[0]) < expected_columns:
                            if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                                # For ADNU tables, assume missing "upgrade" column
                                print(f"Detected missing 'upgrade' column in continuation table on page {page_number + 1}, table {table_index + 1}. Inserting 'upgrade' column.", file=log_file)
                                data_rows = [[specific_phrase] + row for row in data_rows]
                            else:
                                # For other tables, assume missing "Type of Upgrade" column
                                print(f"Detected missing 'Type of Upgrade' column in continuation table on page {page_number + 1}, table {table_index + 1}. Inserting 'Type of Upgrade' column.", file=log_file)
                                data_rows = [["Type of Upgrade"] + row for row in data_rows]
                        elif len(data_rows[0]) > expected_columns:
                            # Extra columns detected; adjust accordingly
                            print(f"Detected extra columns in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping extra columns.", file=log_file)
                            data_rows = [row[:expected_columns] for row in data_rows]

                        # Create DataFrame for the continuation table
                        try:
                            df_continuation = pd.DataFrame(data_rows, columns=expected_headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for continuation table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue  # Skip this table due to error

                        # Special Handling for "Area Delivery Network Upgrade" Tables in Continuation
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            if "upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["upgrade"]) or first_row["upgrade"] == "":
                                    print(f"Replacing 'None' in 'upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "upgrade"] = specific_phrase
                            else:
                                # If "upgrade" column does not exist, add it
                                df_continuation["upgrade"] = specific_phrase
                                print(f"'upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        else:
                            # General Handling for other tables
                            if "type of upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing 'None' in 'Type of Upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "type of upgrade"] = specific_phrase
                            else:
                                # If "Type of Upgrade" column does not exist, add it
                                df_continuation["type of upgrade"] = specific_phrase
                                print(f"'Type of Upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        # Ensure no duplicate columns
                        if df_continuation.columns.duplicated().any():
                            print(f"Duplicate columns detected in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping duplicates.", file=log_file)
                            df_continuation = df_continuation.loc[:, ~df_continuation.columns.duplicated()]

                        # Merge with the last extracted table
                        extracted_tables[-1] = pd.concat([extracted_tables[-1], df_continuation], ignore_index=True, sort=False)

    except Exception as e:
        print(f"Error processing Table 7 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()

    # After scraping all tables, concatenate them
    if extracted_tables:
        # To prevent reindexing errors, ensure all DataFrames have unique and consistent columns
        # Collect all unique column names
        all_columns = set()
        for df in extracted_tables:
            all_columns.update(df.columns.tolist())
        #all_columns = sorted(all_columns)  # Optional: sort columns for consistency

        standardized_tables = []
        for df in extracted_tables:
            # Reindex to have all columns, filling missing ones with NaN
            standardized_df = df.reindex(columns=all_columns)
            standardized_tables.append(standardized_df)

        print("\nConcatenating all extracted Table 7 data...", file=log_file)
        try:
            table7_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
            print(f"Successfully concatenated {len(standardized_tables)} tables.", file=log_file)
        except Exception as e:
            print(f"Error concatenating tables: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            table7_data = pd.DataFrame()
    else:
        print("No Table 7 data extracted.", file=log_file)
        table7_data = pd.DataFrame()

    return table7_data

def extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """Extracts Table 7 data and merges with base data."""
    base_data = extract_base_data(pdf_path, project_id, log_file)
    table7_data = extract_table7(pdf_path, log_file, is_addendum)

    if table7_data.empty:
        return base_data
    else:
        # Identify overlapping columns excluding 'point_of_interconnection'
        overlapping_columns = base_data.columns.intersection(table7_data.columns).difference(['point_of_interconnection'])
        table7_data = table7_data.drop(columns=overlapping_columns, errors='ignore')
        
        # Repeat base data for each row in table7_data
        base_data_repeated = pd.concat([base_data] * len(table7_data), ignore_index=True)
        
        try:
            # Concatenate base data with Table 7 data along columns
            merged_df = pd.concat([base_data_repeated, table7_data], axis=1, sort=False)
            
            # Ensure 'point_of_interconnection' is present and correctly populated
            if 'point_of_interconnection' not in merged_df.columns:
                merged_df['point_of_interconnection'] = base_data['point_of_interconnection'].iloc[0]
                print(f"Added 'point_of_interconnection' to merged data for {pdf_path}.", file=log_file)
            
            print(f"Merged base data with Table 7 data for {pdf_path}.", file=log_file)
            return merged_df
        except Exception as e:
            print(f"Error merging base data with Table 7 data for {pdf_path}: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            return base_data  # Fallback to base data only



            

def check_has_table7(pdf_path):
    """Checks if the PDF contains Table 7."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text() or ""
                if re.search(r"(Modification\s+of\s+)?Table\s*7[-.]?\d*", text, re.IGNORECASE):
                    return True
    except Exception as e:
        # Handle potential errors when opening PDF
        return False
    return False

def is_addendum(pdf_path):
    """Checks if the PDF is an addendum by searching 'Addendum' on the first page."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if len(pdf.pages) == 0:
                return False
            first_page = pdf.pages[0]
            text = first_page.extract_text() or ""
            return "Addendum" in text
    except Exception as e:
        # Handle potential errors when opening PDF
        return False

def process_pdfs_in_folder():
    """Processes all PDFs in the specified project range and directory."""
    global core_originals, core_addendums, total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped

    # Ensure the log file directory exists
    os.makedirs(os.path.dirname(LOG_FILE_PATH), exist_ok=True)

    with open(LOG_FILE_PATH, 'w') as log_file:
        for project_id in PROJECT_RANGE:
            project_path = os.path.join(BASE_DIRECTORY, str(project_id), "02_phase_1_study")
            if not os.path.exists(project_path):
                missing_projects.add(project_id)
                print(f"Project path does not exist: {project_path}", file=log_file)
                continue

            project_scraped = False  # Flag to track if any PDF in the project was scraped
            base_data_extracted = False
            base_data = pd.DataFrame()

            for pdf_name in os.listdir(project_path):
                if pdf_name.endswith(".pdf"):
                    pdf_path = os.path.join(project_path, pdf_name)
                    total_pdfs_accessed += 1
                    is_add = is_addendum(pdf_path)

                    # Determine output DataFrame and CSV path based on addendum status
                    if is_add:
                        addendum_pdfs.append(pdf_name)
                        print(f"Accessing Addendum PDF: {pdf_name} from Project {project_id}", file=log_file)
                    else:
                        original_pdfs.append(pdf_name)
                        print(f"Accessing Original PDF: {pdf_name} from Project {project_id}", file=log_file)

                    try:
                        has_table7 = check_has_table7(pdf_path)
                        if not has_table7:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 7)", file=log_file)
                            # Print to ipynb output
                            print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 7)")
                            total_pdfs_skipped += 1
                            continue

                        if not is_add and not base_data_extracted:
                            # Extract base data from original PDF
                            base_data = extract_base_data(pdf_path, project_id, log_file)
                            base_data_extracted = True
                            print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)

                        if is_add and base_data_extracted:
                            # For addendums, use the extracted base data
                            table7_data = extract_table7(pdf_path, log_file, is_addendum=is_add)
                            if not table7_data.empty:
                                # Merge base data with Table 7 data
                                merged_df = pd.concat([base_data] * len(table7_data), ignore_index=True)
                                merged_df = pd.concat([merged_df, table7_data], axis=1, sort=False)
                                core_addendums = pd.concat([core_addendums, merged_df], ignore_index=True)
                                scraped_pdfs.append(pdf_name)
                                scraped_projects.add(project_id)
                                project_scraped = True
                                total_pdfs_scraped += 1
                                print(f"Scraped Addendum PDF: {pdf_name} from Project {project_id}")
                            else:
                                skipped_pdfs.append(pdf_name)
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                                # Optionally, print to ipynb
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)")
                                total_pdfs_skipped += 1
                        else:
                            # For originals, extract Table 7 data
                            df = extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=is_add)
                            if not df.empty:
                                if is_add:
                                    core_addendums = pd.concat([core_addendums, df], ignore_index=True)
                                else:
                                    core_originals = pd.concat([core_originals, df], ignore_index=True)
                                scraped_pdfs.append(pdf_name)
                                scraped_projects.add(project_id)
                                project_scraped = True
                                total_pdfs_scraped += 1
                                print(f"Scraped PDF: {pdf_name} from Project {project_id}")
                            else:
                                skipped_pdfs.append(pdf_name)
                                print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                                # Optionally, print to ipynb
                                print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)")
                                total_pdfs_skipped += 1

                    except Exception as e:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                        print(traceback.format_exc(), file=log_file)
                        # Optionally, print to ipynb
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                        total_pdfs_skipped += 1

            # After processing all PDFs for this project, check if any PDF was scraped
            if not project_scraped and os.path.exists(project_path):
                skipped_projects.add(project_id)


    # After processing all PDFs, save to CSV
    save_to_csv(core_originals, OUTPUT_CSV_PATH_ORIGINAL, "originals")
    save_to_csv(core_addendums, OUTPUT_CSV_PATH_ADDENDUM, "addendums")

    # Calculate total projects processed
    total_projects_processed = len(scraped_projects) + len(skipped_projects)

    # Print summary to ipynb
    print("\n=== Scraping Summary ===")
    print(f"Total Projects Processed: {total_projects_processed}")
    print(f"Total Projects Scraped: {len(scraped_projects)}")
    print(f"Total Projects Skipped: {len(skipped_projects)}")
    print(f"Total Projects Missing: {len(missing_projects)}")
    print(f"Total PDFs Accessed: {total_pdfs_accessed}")
    print(f"Total PDFs Scraped: {total_pdfs_scraped}")
    print(f"Total PDFs Skipped: {total_pdfs_skipped}")

    print("\nList of Scraped Projects:")
    print(sorted(scraped_projects))

    print("\nList of Skipped Projects:")
    print(sorted(skipped_projects))

    print("\nList of Missing Projects:")
    print(sorted(missing_projects))

    print("\nList of Scraped PDFs:")
    print(scraped_pdfs)

    print("\nList of Skipped PDFs:")
    print(skipped_pdfs)

    print("\nList of Addendum PDFs:")
    print(addendum_pdfs)

    print("\nList of Original PDFs:")
    print(original_pdfs)

    print("\nNumber of Original PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in original_pdfs]))
    print("Number of Addendum PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in addendum_pdfs]))

def save_to_csv(df, output_csv_path, data_type):
    """Cleans the DataFrame and saves it to a CSV file."""
    if df.empty:
        print(f"No data to save for {data_type}.")
        return

    # Clean up the entire DataFrame by cleaning string cells
    df = df.map(clean_string_cell)

    # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
    df = df[~df.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]

    # Reorder columns as specified
    df = reorder_columns(df)
    print(f"\nColumns reordered for {data_type} as per specification.")

    # Ensure q_id is numeric for sorting, replace missing values with None
    if 'q_id' in df.columns:
        df['q_id'] = pd.to_numeric(df['q_id'], errors='coerce')

    # Save the DataFrame to CSV
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"\nData successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving {data_type} data to CSV: {e}")
        print(traceback.format_exc())

def main():
    """Main function to execute the PDF scraping process."""
    process_pdfs_in_folder()

if __name__ == "__main__":
    main()


Scraped PDF: C7PhI Appendix A - Q1007 Coyote Creek Energy Storage.pdf from Project 1007
Scraped PDF: C7PhI Appendix A - Q1010 Dyer Summit Wind Repower.pdf from Project 1010
Scraped PDF: C7PhI Appendix A - Q1011 GHS Project.pdf from Project 1011
Scraped PDF: C7PhI Appendix A - Q1013 Lake Clementine Hydroelectric.pdf from Project 1013
Scraped PDF: C7PhI Appendix A - Q1014 Paradise Cut.pdf from Project 1014
Scraped PDF: C7PhI Appendix A - Q1015 Pleasant Grove - Grid Stabilization Project.pdf from Project 1015
Scraped PDF: C7PhI Appendix A - Q1016 Richmond Steam Turbine.pdf from Project 1016
Scraped PDF: C7PhI Appendix A - Q1018 SGS South San Francisco.pdf from Project 1018
Scraped PDF: C7PhI Appendix A - Q1019 Shingle Springs - Grid Stabilization Project.pdf from Project 1019
Scraped PDF: C7PhI Appendix A - Q1020 Sky Valley Wind.pdf from Project 1020
Scraped PDF: C7PhI Appendix A - Q1021 Stockton Biomass.pdf from Project 1021
Scraped Addendum PDF: Q1021_StocktonBiomass_PHI_Report-Addendum

# Merge Original and Addendums 

In [9]:
import pandas as pd
import re

# Load original and addendum data
df_original = pd.read_csv(
    '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/rawdata_cluster7_style_G_originals.csv',
    dtype={'estimated_time_to_construct': str}
)
df_addendum = pd.read_csv(
    '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/rawdata_cluster7_style_G_addendums.csv',
    dtype={'estimated_time_to_construct': str}
)

# Step 1: Align column names and merge similar columns
def merge_columns(df):
    merge_columns_dict = {
        "escalated_cost_x_1000": [
            "escalated costs x 1000",
            "estimated cost x 1000 escalated",
            "estimated cost x 1000 escalated with itcca"
        ],
        "estimated_time_to_construct": [
            "estimated time to construct",
            "estimated time  to construct"
        ],
        "total_estimated_cost_x_1000_escalated": [
            "total estimated cost x 1000 escalalted",
            "total estimated cost x 1000 escalated"
        ],
        "adnu_cost_rate_x_1000": [
            "adnu cost rate x 1000",
            "cost rate x 1000"
        ],
        "description": ["description"],
        "capacity": [
            "capacity",
            "project size"
        ],
        "cost_allocation_factor": [
            "cost allocation factor",
            "cost allocatio n factor"
        ],
    }

    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]

    if unnamed_columns:
        merge_columns_dict["description"].extend(unnamed_columns)

    for new_col, old_cols in merge_columns_dict.items():
        existing_cols = [col for col in old_cols if col in df.columns]
        if existing_cols:
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
            df.drop(columns=[col for col in existing_cols if col != new_col], inplace=True)
    return df

def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

df_original = merge_columns(df_original)
df_addendum = merge_columns(df_addendum)

# Convert column names to snake_case
df_original.columns = [convert_to_snake_case(col) for col in df_original.columns]
df_addendum.columns = [convert_to_snake_case(col) for col in df_addendum.columns]

# Step 2: Replace and merge data using group-based approach

# Identify the key columns
key_columns = ['q_id', 'type_of_upgrade', 'upgrade']

# Ensure key columns exist in both dataframes
for col in key_columns:
    if col not in df_original.columns or col not in df_addendum.columns:
        raise ValueError(f"Key column '{col}' is missing from one of the dataframes.")

# Identify (q_id, type_of_upgrade) pairs present in addendums
addendum_groups = df_addendum[['q_id', 'type_of_upgrade']].drop_duplicates()

# Filter out rows from the original that have (q_id, type_of_upgrade) present in addendums
original_filtered = df_original.merge(
    addendum_groups,
    on=['q_id', 'type_of_upgrade'],
    how='left',
    indicator=True
)
original_filtered = original_filtered[original_filtered['_merge'] == 'left_only'].drop(columns=['_merge'])

# Concatenate the filtered original with addendums
df_updated = pd.concat([original_filtered, df_addendum], ignore_index=True)

# Optional: Sort the updated dataframe based on keys for better readability
df_updated.sort_values(by=key_columns, inplace=True)
df_updated.reset_index(drop=True, inplace=True)

# Step 3: Save the updated file
updated_file = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/rawdata_cluster7_style_G_updated.csv'
df_updated.to_csv(updated_file, index=False)
print("Updated file saved successfully.")


Updated file saved successfully.


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_24577/2448823523.py:53: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]


# Cleaning Code to create itemized and total dataset

# Originals

In [14]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
# df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 6-8/C_6_7_8_data.csv', dtype={'estimated_time_to_construct': str})

df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/rawdata_cluster7_style_G_originals.csv', dtype={'estimated_time_to_construct': str})

df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


def merge_columns(df):
    merge_columns_dict = {
        "escalated_cost_x_1000": [
            "escalated costs x 1000",
            "estimated cost x 1000 escalated",
            "estimated cost x 1000 escalated with itcca"
        ],
        "estimated_time_to_construct": [
            "estimated time to construct",
            "estimated time  to construct"
        ],
        "total_estimated_cost_x_1000_escalated": [
            "total estimated cost x 1000 escalalted",
            "total estimated cost x 1000 escalated"
        ],
        "adnu_cost_rate_x_1000": [
            "adnu cost rate x 1000",
            "cost rate x 1000"
        ],
        "description": ["description"],
        "capacity": [
            "capacity",
            "project size"
        ],
        "cost_allocation_factor": [
            "cost allocation factor",
            "cost allocatio n factor"
        ],
    }

    # Identify unnamed columns (columns with no name or those starting with "Unnamed")
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]

    if unnamed_columns:
        # Merge unnamed columns into 'description' if they exist
        merge_columns_dict["description"].extend(unnamed_columns)

    # Iterate over the dictionary to merge columns
    for new_col, old_cols in merge_columns_dict.items():
        # Get the list of columns that exist in the DataFrame
        existing_cols = [col for col in old_cols if col in df.columns]

        if existing_cols:
            # Create a new column by backfilling data from the existing old columns
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]

            # Drop the old columns, except for the new one we just created
            cols_to_drop = [col for col in existing_cols if col != new_col]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

    return df


# Apply the merge_columns function to the DataFrame
df = merge_columns(df)


def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name


def clean_string_cell(value):
    if isinstance(value, str):
        # Normalize Unicode to NFKD and remove non-ASCII characters
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        # Remove newlines and extra spaces
        value = value.replace('\n', ' ').strip()
    return value


# Apply the clean_string_cell function to all cells using applymap
df = df.map(clean_string_cell)


# Apply the function to all columns to convert column names to snake_case
df.columns = [convert_to_snake_case(col) for col in df.columns]


# Ensure that the necessary columns exist before proceeding
required_columns = ['type_of_upgrade', 'cost_allocation_factor']
for col in required_columns:
    if col not in df.columns:
        df[col] = None  # or any default value you prefer


# Step 1: Create the 'item' column based on whether the 'type_of_upgrade' row or 'cost_allocation_factor' contains 'Total'
df['item'] = df.apply(
    lambda row: 'no' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) or
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor']))
    ) else 'yes',
    axis=1
)

# Step 2: Move 'item' column next to 'type_of_upgrade' column if both exist
if 'item' in df.columns and 'type_of_upgrade' in df.columns:
    cols = df.columns.tolist()
    item_index = cols.index('item')
    type_index = cols.index('type_of_upgrade')
    if item_index < type_index:
        cols.insert(type_index + 1, cols.pop(item_index))
    else:
        cols.insert(type_index + 1, cols.pop(item_index))
    df = df[cols]

# Step 3: Remove the 'Total' values from the 'cost_allocation_factor' column if they are already in the 'type_of_upgrade' column
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if (
            pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])
        ) else row.get('cost_allocation_factor'),
        axis=1
    )

# Step 4: For each `q_id` and `type_of_upgrade`, if it has only one row and no 'Total' is present in 'cost_allocation_factor', create a new `Total` row
new_rows = []
for q_id, group in df.groupby('q_id'):
    unique_upgrades = group['type_of_upgrade'].dropna().unique()

    # Check if there's already a "Total" in 'cost_allocation_factor' for this q_id
    if any('Total' in str(x) for x in group.get('cost_allocation_factor', [])):
        continue  # If Total exists under 'cost_allocation_factor', skip creating new total row for this q_id

    for upgrade in unique_upgrades:
        # Skip if "Total" is already present for this upgrade or if the upgrade is NaN
        if pd.isna(upgrade) or 'Total' in str(upgrade):
            continue

        # Get rows corresponding to this specific upgrade
        rows = group[group['type_of_upgrade'] == upgrade]

        if len(rows) == 1:
            # Duplicate the row and create a new row with "Total" under 'type_of_upgrade'
            original_row = rows.iloc[0].copy()
            total_row = original_row.copy()

            # Modify the total_row to reflect "Total"
            total_row['type_of_upgrade'] = 'Total'
            total_row['item'] = 'no'

            # Append the Total row immediately after the original row
            original_index = df[(df['q_id'] == q_id) & (df['type_of_upgrade'] == upgrade)].index[0]
            new_rows.append((original_index + 1, total_row))

# Step 5: Insert the new rows into the DataFrame in the correct order
for idx, row in sorted(new_rows, reverse=True):
    if idx < 0 or idx > len(df):
        continue  # Skip invalid indices
    df = pd.concat([df.iloc[:idx], pd.DataFrame([row]), df.iloc[idx:]]).reset_index(drop=True)

# Step 6: Move "Total" from 'cost_allocation_factor' column to 'type_of_upgrade' column
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None

    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

# Step 7: After moving "Total" to 'type_of_upgrade', clear it from 'cost_allocation_factor'
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if 'Total' in str(row.get('cost_allocation_factor', '')) else row.get('cost_allocation_factor'),
        axis=1
    )

'''
def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'\s*months?\s*', '', value, flags=re.IGNORECASE).strip()
    return value
'''

def clean_estimated_time(value):
    if isinstance(value, str):
        # Remove any word after the number, preserving hyphens
        # This will strip words like 'months', 'years', etc., but keep hyphens
        value = re.sub(r'(\d+(?:-\w+)*)\s+\w+.*$', r'\1', value, flags=re.IGNORECASE).strip()
    return value


if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )


mappings = {
    'Reliability Network Upgrade': 'RNU',
    'Reliability Network upgrade': 'RNU',
    'Local Delivery Network Upgrade': 'LDNU',
    'Local Delivery Network': 'LDNU',
    "PTOs Interconnection Facilities": 'PTO_IF',
    'Area Delivery Network Upgrade': 'ADNU',
    'Reliability Network upgrade to Physically Interconnect': 'RNU',
    'PTO': 'PTO_IF',
}

# Apply transformations using a lambda function
if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )
    # Step 8: Forward fill the non-empty values in 'type_of_upgrade' to replace empty cells with the previous value
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()


if 'upgrade' in df.columns and 'type_of_upgrade' in df.columns and 'q_id' in df.columns:
    df['upgrade'] = df.groupby(['q_id', 'type_of_upgrade'])['upgrade'].ffill()


# Step 9: Ensure 'Total' is correctly replaced in 'type_of_upgrade' when present
if 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None

    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

# Step 10: Fill NaN in descriptive columns with 'None' and NaN in numeric columns with 0
# List of numeric columns and non-numeric columns based on dataset structure
numeric_columns = [
    'cost_allocation_factor',
    'estimated_cost_x_1000',
    'estimated_time_to_construct',
    'total_estimated_cost_x_1000_escalated',
    'adnu_cost_rate_x_1000',
    'escalated_cost_x_1000',
    'estimated_cost_x_1000_escalated_without_itcca',
    'adnu_cost_rate_x_1000_escalated'
]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

# Replace '-' with NaN in numeric columns (so they can be converted to 0)
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].replace('-', pd.NA)

# Replace NaN in numeric columns with 0
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Step 11: Sort the DataFrame by q_id and original order if 'original_order' exists
if 'original_order' in df.columns and 'q_id' in df.columns:
    df['original_order'] = df.index
    df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])
    # Optional: Drop the original_order column if not needed anymore
    df = df.drop(columns=['original_order'])

# Step 12: Create the itemized dataset (rows where item == 'yes') and save to CSV
if 'item' in df.columns:
    itemized_df = df[df['item'] == 'yes']
    itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_itemized.csv', index=False)

# Step 13: Create the totals dataset (rows where item == 'no') and drop unwanted columns
totals_columns = ['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct']
existing_totals_columns = [col for col in totals_columns if col in df.columns]
if 'item' in df.columns:
    totals_df = df[df['item'] == 'no'].drop(columns=existing_totals_columns, errors='ignore')
    totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_total.csv', index=False)

print(f"Itemized rows saved to 'costs_phase_1_cluster_7_itemized.csv'.")
print(f"Filtered Total rows saved to 'costs_phase_1_cluster_7_total.csv'.")

# Print unique values if columns exist
if 'type_of_upgrade' in df.columns:
    print(df['type_of_upgrade'].unique())

if 'q_id' in df.columns:
    print(df['q_id'].unique())

if 'cluster' in df.columns:
    print(df['cluster'].unique())


Itemized rows saved to 'costs_phase_1_cluster_7_itemized.csv'.
Filtered Total rows saved to 'costs_phase_1_cluster_7_total.csv'.
['PTO_IF' 'RNU' 'LDNU' 'ADNU']
[1007 1010 1011 1013 1014 1015 1016 1018 1019 1020 1021 1023 1024 1026
 1027 1028 1029 1030 1031 1032 1033 1035 1036 1037 1038 1040 1043 1045
 1046 1047 1048 1049 1050 1051 1052 1053 1055 1056 1057 1058 1059 1060
 1061 1062 1063]
[7]


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_75037/408863846.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]


# Addendums

In [21]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
# df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 6-8/C_6_7_8_data.csv', dtype={'estimated_time_to_construct': str})

df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/03_raw/rawdata_cluster7_style_G_addendums.csv', dtype={'estimated_time_to_construct': str})

df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


def merge_columns(df):
    merge_columns_dict = {
        "escalated_cost_x_1000": [
            "escalated costs x 1000",
            "estimated cost x 1000 escalated",
            "estimated cost x 1000 escalated with itcca"
        ],
        "estimated_time_to_construct": [
            "estimated time to construct",
            "estimated time  to construct",
            "estimated time"
        ],
        "total_estimated_cost_x_1000_escalated": [
            "total estimated cost x 1000 escalalted",
            "total estimated cost x 1000 escalated"
        ],
        "adnu_cost_rate_x_1000": [
            "adnu cost rate x 1000",
            "cost rate x 1000"
        ],
        "description": ["description"],
        "capacity": [
            "capacity",
            "project size"
        ],
        "cost_allocation_factor": [
            "cost allocation factor",
            "cost allocatio n factor"
        ],
    }

    # Identify unnamed columns (columns with no name or those starting with "Unnamed")
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]

    if unnamed_columns:
        # Merge unnamed columns into 'description' if they exist
        merge_columns_dict["description"].extend(unnamed_columns)

    # Iterate over the dictionary to merge columns
    for new_col, old_cols in merge_columns_dict.items():
        # Get the list of columns that exist in the DataFrame
        existing_cols = [col for col in old_cols if col in df.columns]

        if existing_cols:
            # Create a new column by backfilling data from the existing old columns
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]

            # Drop the old columns, except for the new one we just created
            cols_to_drop = [col for col in existing_cols if col != new_col]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

    return df


# Apply the merge_columns function to the DataFrame
df = merge_columns(df)


def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name


def clean_string_cell(value):
    if isinstance(value, str):
        # Normalize Unicode to NFKD and remove non-ASCII characters
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        # Remove newlines and extra spaces
        value = value.replace('\n', ' ').strip()
    return value


# Apply the clean_string_cell function to all cells using applymap
df = df.map(clean_string_cell)


# Apply the function to all columns to convert column names to snake_case
df.columns = [convert_to_snake_case(col) for col in df.columns]


# Ensure that the necessary columns exist before proceeding
required_columns = ['type_of_upgrade', 'cost_allocation_factor']
for col in required_columns:
    if col not in df.columns:
        df[col] = None  # or any default value you prefer


# Step 1: Create the 'item' column based on whether the 'type_of_upgrade' row or 'cost_allocation_factor' contains 'Total'
df['item'] = df.apply(
    lambda row: 'no' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) or
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor']))
    ) else 'yes',
    axis=1
)

# Step 2: Move 'item' column next to 'type_of_upgrade' column if both exist
if 'item' in df.columns and 'type_of_upgrade' in df.columns:
    cols = df.columns.tolist()
    item_index = cols.index('item')
    type_index = cols.index('type_of_upgrade')
    if item_index < type_index:
        cols.insert(type_index + 1, cols.pop(item_index))
    else:
        cols.insert(type_index + 1, cols.pop(item_index))
    df = df[cols]

# Step 3: Remove the 'Total' values from the 'cost_allocation_factor' column if they are already in the 'type_of_upgrade' column
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if (
            pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])
        ) else row.get('cost_allocation_factor'),
        axis=1
    )

# Step 4: For each `q_id` and `type_of_upgrade`, if it has only one row and no 'Total' is present in 'cost_allocation_factor', create a new `Total` row
new_rows = []
for q_id, group in df.groupby('q_id'):
    unique_upgrades = group['type_of_upgrade'].dropna().unique()

    # Check if there's already a "Total" in 'cost_allocation_factor' for this q_id
    if any('Total' in str(x) for x in group.get('cost_allocation_factor', [])):
        continue  # If Total exists under 'cost_allocation_factor', skip creating new total row for this q_id

    for upgrade in unique_upgrades:
        # Skip if "Total" is already present for this upgrade or if the upgrade is NaN
        if pd.isna(upgrade) or 'Total' in str(upgrade):
            continue

        # Get rows corresponding to this specific upgrade
        rows = group[group['type_of_upgrade'] == upgrade]

        if len(rows) == 1:
            # Duplicate the row and create a new row with "Total" under 'type_of_upgrade'
            original_row = rows.iloc[0].copy()
            total_row = original_row.copy()

            # Modify the total_row to reflect "Total"
            total_row['type_of_upgrade'] = 'Total'
            total_row['item'] = 'no'

            # Append the Total row immediately after the original row
            original_index = df[(df['q_id'] == q_id) & (df['type_of_upgrade'] == upgrade)].index[0]
            new_rows.append((original_index + 1, total_row))

# Step 5: Insert the new rows into the DataFrame in the correct order
for idx, row in sorted(new_rows, reverse=True):
    if idx < 0 or idx > len(df):
        continue  # Skip invalid indices
    df = pd.concat([df.iloc[:idx], pd.DataFrame([row]), df.iloc[idx:]]).reset_index(drop=True)

# Step 6: Move "Total" from 'cost_allocation_factor' column to 'type_of_upgrade' column
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None

    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

# Step 7: After moving "Total" to 'type_of_upgrade', clear it from 'cost_allocation_factor'
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if 'Total' in str(row.get('cost_allocation_factor', '')) else row.get('cost_allocation_factor'),
        axis=1
    )

'''
def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'\s*months?\s*', '', value, flags=re.IGNORECASE).strip()
    return value
'''

def clean_estimated_time(value):
    if isinstance(value, str):
        # Remove any word after the number, preserving hyphens
        # This will strip words like 'months', 'years', etc., but keep hyphens
        value = re.sub(r'(\d+(?:-\w+)*)\s+\w+.*$', r'\1', value, flags=re.IGNORECASE).strip()
    return value


if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )


mappings = {
    'Reliability Network Upgrade': 'RNU',
    'Reliability Network upgrade': 'RNU',
    'Local Delivery Network Upgrade': 'LDNU',
    'Local Delivery Network': 'LDNU',
    "PTOs Interconnection Facilities": 'PTO_IF',
    'Area Delivery Network Upgrade': 'ADNU',
    'Reliability Network upgrade to Physically Interconnect': 'RNU',
    'PTO': 'PTO_IF',
}

# Apply transformations using a lambda function
if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )
    # Step 8: Forward fill the non-empty values in 'type_of_upgrade' to replace empty cells with the previous value
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()


if 'upgrade' in df.columns and 'type_of_upgrade' in df.columns and 'q_id' in df.columns:
    df['upgrade'] = df.groupby(['q_id', 'type_of_upgrade'])['upgrade'].ffill()


# Step 9: Ensure 'Total' is correctly replaced in 'type_of_upgrade' when present
if 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None

    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

# Step 10: Fill NaN in descriptive columns with 'None' and NaN in numeric columns with 0
# List of numeric columns and non-numeric columns based on dataset structure
numeric_columns = [
    'cost_allocation_factor',
    'estimated_cost_x_1000',
    'estimated_time_to_construct',
    'total_estimated_cost_x_1000_escalated',
    'adnu_cost_rate_x_1000',
    'escalated_cost_x_1000',
    'estimated_cost_x_1000_escalated_without_itcca',
    'adnu_cost_rate_x_1000_escalated'
]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

# Replace '-' with NaN in numeric columns (so they can be converted to 0)
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].replace('-', pd.NA)

# Replace NaN in numeric columns with 0
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Step 11: Sort the DataFrame by q_id and original order if 'original_order' exists
if 'original_order' in df.columns and 'q_id' in df.columns:
    df['original_order'] = df.index
    df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])
    # Optional: Drop the original_order column if not needed anymore
    df = df.drop(columns=['original_order'])

# Step 12: Create the itemized dataset (rows where item == 'yes') and save to CSV
if 'item' in df.columns:
    itemized_df = df[df['item'] == 'yes']
    itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_itemized_addendums.csv', index=False)

# Step 13: Create the totals dataset (rows where item == 'no') and drop unwanted columns
totals_columns = ['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct']
existing_totals_columns = [col for col in totals_columns if col in df.columns]
if 'item' in df.columns:
    totals_df = df[df['item'] == 'no'].drop(columns=existing_totals_columns, errors='ignore')
    totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_total_addendums.csv', index=False)

print(f"Itemized rows saved to 'costs_phase_1_cluster_7_itemized.csv'.")
print(f"Filtered Total rows saved to 'costs_phase_1_cluster_7_total.csv'.")

# Print unique values if columns exist
if 'type_of_upgrade' in df.columns:
    print(df['type_of_upgrade'].unique())

if 'q_id' in df.columns:
    print(df['q_id'].unique())

if 'cluster' in df.columns:
    print(df['cluster'].unique())


Itemized rows saved to 'costs_phase_1_cluster_7_itemized.csv'.
Filtered Total rows saved to 'costs_phase_1_cluster_7_total.csv'.
['PTO_IF' 'RNU']
[1021]
[7]


# Code to merge the itemized and total datasets for original and addendum files

In [38]:
import pandas as pd

def load_data(file_path, char_columns):
    """
    Load a CSV file and ensure specific columns are treated as character, others as numeric.
    """
 # Get columns available in the dataset
    available_columns = pd.read_csv(file_path, nrows=0).columns
    
    # Restrict to char_columns that are present in the dataset
    char_columns_in_dataset = [col for col in char_columns if col in available_columns]
    
    # Load the dataset, treating char_columns_in_dataset as strings
    df = pd.read_csv(
        file_path,
        dtype={col: str for col in char_columns_in_dataset},
        na_values=[],  # Disable automatic NaN interpretation
        keep_default_na=False  # Prevent treating "None" as NaN
    )

    
    
    
    # Convert all other columns to numeric
    #for col in df.columns:
    #    if col not in char_columns:
    #        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
    
    return df

def save_data(df, file_path, char_columns):
    """
    Save a dataframe to a CSV file, ensuring specific columns are treated as character.
    """
    for col in char_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)
    df.to_csv(file_path, index=False)

def merge_with_addendums(itemized, itemized_addendums, total, total_addendums):
    # Add an 'original' column to the datasets
    itemized['original'] = "yes"
    total['original'] = "yes"
    
    # Preserve the original row order
    itemized['row_order'] = itemized.index
    total['row_order'] = total.index
    
    # Prepare a list to collect the updated itemized rows
    updated_itemized_rows = []
    
    # Merge itemized and itemized_addendums
    for q_id in itemized_addendums['q_id'].unique():
        for upgrade_type in itemized_addendums['type_of_upgrade'].unique():
            addendum_rows = itemized_addendums[
                (itemized_addendums['q_id'] == q_id) &
                (itemized_addendums['type_of_upgrade'] == upgrade_type)
            ]
            if not addendum_rows.empty:
                mask = (itemized['q_id'] == q_id) & (itemized['type_of_upgrade'] == upgrade_type)
                original_rows = itemized[mask]

                # Find non-character columns missing in the addendum dataset
                missing_cols = set(itemized.columns) - set(addendum_rows.columns) - set(char_columns)
                for col in missing_cols:
                    addendum_rows[col] = 0 
                itemized.loc[mask, 'original'] = "no"
                updated_itemized_rows.append(addendum_rows.assign(original="no", row_order=original_rows['row_order'].values[:len(addendum_rows)]))
                itemized = itemized[~mask]

            
    
    if updated_itemized_rows:
        updated_itemized = pd.concat([itemized] + updated_itemized_rows, ignore_index=True)
    else:
        updated_itemized = itemized.copy()
    
    updated_itemized = updated_itemized.sort_values(by="row_order").drop(columns=["row_order"]).reset_index(drop=True)
    
    updated_total_rows = []
    
    for q_id in total_addendums['q_id'].unique():
        for upgrade_type in total_addendums['type_of_upgrade'].unique():
            addendum_row = total_addendums[
                (total_addendums['q_id'] == q_id) &
                (total_addendums['type_of_upgrade'] == upgrade_type)
            ]
            if not addendum_row.empty:
                mask = (total['q_id'] == q_id) & (total['type_of_upgrade'] == upgrade_type)
                original_row = total[mask]

                # Find non-character columns missing in the addendum dataset
                missing_cols = set(itemized.columns) - set(addendum_rows.columns) - set(char_columns)
                for col in missing_cols:
                    addendum_rows[col] = 0 
                total.loc[mask, 'original'] = "no"
                updated_total_rows.append(addendum_row.assign(original="no", row_order=original_row['row_order'].values[:len(addendum_row)]))
                total = total[~mask]
    
    if updated_total_rows:
        updated_total = pd.concat([total] + updated_total_rows, ignore_index=True)
    else:
        updated_total = total.copy()
    
    updated_total = updated_total.sort_values(by="row_order").drop(columns=["row_order"]).reset_index(drop=True)
    
    # Fill missing columns with zeros in the updated datasets
    for col in set(itemized.columns) - set(updated_itemized.columns):
        updated_itemized[col] = 0
    
    for col in set(total.columns) - set(updated_total.columns):
        updated_total[col] = 0



    
    # Move the 'original' column to the last position
    updated_itemized = updated_itemized[[col for col in updated_itemized.columns if col != 'original'] + ['original']]
    updated_total = updated_total[[col for col in updated_total.columns if col != 'original'] + ['original']]

        # Drop the row_order column directly
    if "row_order" in updated_itemized.columns:
        updated_itemized = updated_itemized.drop(columns=["row_order"]).reset_index(drop=True)

    if "row_order" in updated_total.columns:
        updated_total = updated_total.drop(columns=["row_order"]).reset_index(drop=True)


  
 
    
    
    return updated_itemized, updated_total

# Define the character columns
char_columns = [
    "req_deliverability", "point_of_interconnection", "type_of_upgrade",
    "upgrade", "description", "estimated_time_to_construct", "original", "item"
]




itemized = load_data("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_itemized.csv", char_columns)
itemized_addendums = load_data("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_itemized_addendums.csv", char_columns)
total = load_data("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_total.csv", char_columns)
total_addendums = load_data("/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_total_addendums.csv", char_columns)


updated_itemized, updated_total = merge_with_addendums(itemized, itemized_addendums, total, total_addendums)


# Save the results
save_data(updated_itemized, "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/01_clean/costs_phase_1_cluster_7_style_G_itemized_updated.csv", char_columns)
save_data(updated_total, "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/01_clean/costs_phase_1_cluster_7_style_G_total_updated.csv", char_columns)


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_24577/2689210007.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addendum_rows[col] = 0
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_24577/2689210007.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addendum_rows[col] = 0
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_24577/2689210007.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

# Checking Scraped Data

In [32]:
import pandas as pd
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# ---------------------- Configuration ---------------------- #

# Paths to the CSV files
ITEMIZED_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_itemized.csv'
TOTALS_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/02_intermediate/costs_phase_1_cluster_7_style_G_total.csv'

# Columns in totals_df that hold the reported total costs
TOTALS_ESTIMATED_COLUMN = 'estimated_cost_x_1000'
TOTALS_ESCALATED_COLUMN = 'escalated_cost_x_1000'

# Upgrade types to check
REQUIRED_UPGRADES = ['PTO_IF', 'RNU', 'LDNU', 'ADNU']

# Output paths
MISMATCHES_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/mismatches.csv'
MATCHED_QIDS_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/matched_qids.csv'

# ---------------------- Load Data ---------------------- #

def load_csv(path, dataset_name):
    """
    Loads a CSV file into a pandas DataFrame.
    """
    try:
        df = pd.read_csv(path)
        print(f"Loaded {dataset_name} from {path}")
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {path}")
        exit(1)
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")
        exit(1)

# Load datasets
itemized_df = load_csv(ITEMIZED_CSV_PATH, "itemized data")
totals_df = load_csv(TOTALS_CSV_PATH, "totals data")

# ---------------------- Data Cleaning ---------------------- #

def clean_text(df, column):
    """
    Cleans text data by stripping leading/trailing spaces and converting to uppercase.
    """
    if column in df.columns:
        df[column] = df[column].astype(str).str.strip().str.upper()
    else:
        print(f"Warning: '{column}' column is missing in the dataset. Filling with 'UNKNOWN'.")
        df[column] = 'UNKNOWN'
    return df

# Clean 'type_of_upgrade' and 'point_of_interconnection' in both datasets
itemized_df = clean_text(itemized_df, 'type_of_upgrade')
itemized_df = clean_text(itemized_df, 'point_of_interconnection')

totals_df = clean_text(totals_df, 'type_of_upgrade')
totals_df = clean_text(totals_df, 'point_of_interconnection')

# ---------------------- Data Preparation ---------------------- #

# Ensure necessary columns exist in itemized_df
required_itemized_columns = ['q_id', 'type_of_upgrade', 'point_of_interconnection', 'estimated_cost_x_1000', 'escalated_cost_x_1000']
for col in required_itemized_columns:
    if col not in itemized_df.columns:
        print(f"Warning: '{col}' column is missing in the itemized dataset.")
        if col in ['q_id', 'type_of_upgrade', 'point_of_interconnection']:
            itemized_df[col] = 'UNKNOWN'
        else:
            itemized_df[col] = 0

# Ensure necessary columns exist in totals_df
required_totals_columns = ['q_id', 'type_of_upgrade', 'point_of_interconnection', TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]
for col in required_totals_columns:
    if col not in totals_df.columns:
        print(f"Error: '{col}' column is missing in the totals dataset. Cannot proceed.")
        exit(1)

# Convert cost columns to numeric, coercing errors to NaN and filling with 0
cost_columns_itemized = ['estimated_cost_x_1000', 'escalated_cost_x_1000']
for col in cost_columns_itemized:
    itemized_df[col] = pd.to_numeric(itemized_df[col], errors='coerce').fillna(0)

cost_columns_totals = [TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]
for col in cost_columns_totals:
    totals_df[col] = pd.to_numeric(totals_df[col], errors='coerce').fillna(0)

# ---------------------- Calculate Manual Totals ---------------------- #

# Group itemized data by q_id and type_of_upgrade and calculate sums
itemized_grouped = itemized_df.groupby(['q_id', 'type_of_upgrade']).agg({
    'estimated_cost_x_1000': 'sum',
    'escalated_cost_x_1000': 'sum'
}).reset_index()

# Apply preference: Use estimated_cost_x_1000 if sum > 0, else use escalated_cost_x_1000
itemized_grouped['manual_total'] = itemized_grouped.apply(
    lambda row: row['estimated_cost_x_1000'] if row['estimated_cost_x_1000'] > 0 else row['escalated_cost_x_1000'],
    axis=1
)

# ---------------------- Prepare Totals Data ---------------------- #

# Group totals data by q_id and type_of_upgrade and calculate sums
totals_grouped = totals_df.groupby(['q_id', 'type_of_upgrade']).agg({
    TOTALS_ESTIMATED_COLUMN: 'sum',
    TOTALS_ESCALATED_COLUMN: 'sum'
}).reset_index()

# Apply preference: Use estimated_cost_x_1000 if sum > 0, else use escalated_cost_x_1000
totals_grouped['reported_total'] = totals_grouped.apply(
    lambda row: row[TOTALS_ESTIMATED_COLUMN] if row[TOTALS_ESTIMATED_COLUMN] > 0 else row[TOTALS_ESCALATED_COLUMN],
    axis=1
)

# ---------------------- Merge Data ---------------------- #

# Merge the itemized and totals data on q_id and type_of_upgrade
comparison_df = pd.merge(
    itemized_grouped,
    totals_grouped[['q_id', 'type_of_upgrade', 'reported_total']],
    on=['q_id', 'type_of_upgrade'],
    how='left'
)

# ---------------------- Check for Missing Upgrades ---------------------- #

# Identify q_ids that are missing any of the required upgrades
missing_upgrades_report = []
for q_id in comparison_df['q_id'].unique():
    upgrades_present = comparison_df[comparison_df['q_id'] == q_id]['type_of_upgrade'].unique()
    missing_upgrades = [upgrade for upgrade in REQUIRED_UPGRADES if upgrade not in upgrades_present]
    if missing_upgrades:
        missing_upgrades_report.append((q_id, missing_upgrades))

# Report missing upgrades
if missing_upgrades_report:
    print("\nQ_ids with missing upgrades:")
    for q_id, missing in missing_upgrades_report:
        print(f"  Q_id {q_id} is missing upgrades: {', '.join(missing)}")
else:
    print("\nAll q_ids have all required upgrades.")

# ---------------------- Compare Totals and Identify Mismatches ---------------------- #

# Initialize list to store mismatches
mismatches = []

# Iterate through each row to compare manual_total with reported_total
for index, row in comparison_df.iterrows():
    q_id = row['q_id']
    upgrade = row['type_of_upgrade']
    manual_total = row['manual_total']
    reported_total = row['reported_total']
    
    # Determine if both manual_total and reported_total are zero
    if manual_total == 0.0 and reported_total == 0.0:
        continue  # No mismatch
    # Determine if manual_total is zero and reported_total is missing or zero
    elif manual_total == 0.0 and (pd.isna(row['reported_total']) or reported_total == 0.0):
        continue  # No mismatch
    # If reported_total is missing (NaN) and manual_total is not zero
    elif pd.isna(row['reported_total']) and manual_total != 0.0:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: Missing")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': 'Missing'
        })
    # If manual_total is not zero and reported_total is zero
    elif manual_total != 0.0 and reported_total == 0.0:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: 0.0")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': reported_total
        })
    # If both totals are non-zero but differ beyond tolerance
    elif abs(manual_total - reported_total) > 1e-2:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: {reported_total}")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': reported_total
        })
    # Else, totals match; do nothing

# Create a DataFrame for mismatches
mismatches_df = pd.DataFrame(mismatches, columns=['q_id', 'type_of_upgrade', 'manual_total', 'reported_total'])

# Save mismatches to a CSV file
try:
    mismatches_df.to_csv(MISMATCHES_CSV_PATH, index=False)
    print(f"\nMismatches saved to '{MISMATCHES_CSV_PATH}'.")
except Exception as e:
    print(f"Error saving mismatches CSV: {e}")

# ---------------------- Point of Interconnection Matching ---------------------- #

# Extract unique q_id and point_of_interconnection from itemized dataset
itemized_poi = itemized_df[['q_id', 'point_of_interconnection']].drop_duplicates()

# Extract unique q_id and point_of_interconnection from totals dataset
totals_poi = totals_df[['q_id', 'point_of_interconnection']].drop_duplicates()

# Merge both to have a complete list of q_id and point_of_interconnection
all_poi = pd.concat([itemized_poi, totals_poi]).drop_duplicates().reset_index(drop=True)

# ---------------------- Direct Match Identification ---------------------- #

# Group by point_of_interconnection to find q_ids sharing the same point_of_interconnection
direct_matches = all_poi.groupby('point_of_interconnection')['q_id'].apply(list).reset_index()

# Filter groups with more than one q_id (i.e., shared points_of_interconnection)
direct_matches = direct_matches[direct_matches['q_id'].apply(len) > 1]

print("\nDirect Matches (Exact Point of Interconnection Names):")
if not direct_matches.empty:
    print(direct_matches)
else:
    print("No direct matches found.")

# ---------------------- Fuzzy Match Identification ---------------------- #

# Prepare list of points_of_interconnection for fuzzy matching
poi_list = all_poi['point_of_interconnection'].unique().tolist()

# Initialize list to store fuzzy matches
fuzzy_matches = []

# Iterate through each point_of_interconnection to find similar ones
for i, poi in enumerate(poi_list):
    # Compare with the rest of the points to avoid redundant comparisons
    similar_pois = process.extract(poi, poi_list[i+1:], scorer=fuzz.token_set_ratio)
    
    # Filter matches with similarity >= 80%
    for match_poi, score in similar_pois:
        if score >= 80:
            # Retrieve q_ids for both points_of_interconnection
            qids_poi1 = all_poi[all_poi['point_of_interconnection'] == poi]['q_id'].tolist()
            qids_poi2 = all_poi[all_poi['point_of_interconnection'] == match_poi]['q_id'].tolist()
            
            # Append the matched pairs with their points_of_interconnection and similarity score
            fuzzy_matches.append({
                'point_of_interconnection_1': poi,
                'q_ids_1': qids_poi1,
                'point_of_interconnection_2': match_poi,
                'q_ids_2': qids_poi2,
                'similarity_score': score
            })

# Convert fuzzy matches to DataFrame
fuzzy_matches_df = pd.DataFrame(fuzzy_matches)

print("\nFuzzy Matches (>=80% Similarity in Point of Interconnection):")
if not fuzzy_matches_df.empty:
    print(fuzzy_matches_df)
else:
    print("No fuzzy matches found.")

# ---------------------- Save Matched Q_ids to CSV ---------------------- #

# For clarity, create a combined DataFrame for direct and fuzzy matches

# Direct matches: list each pair of q_ids sharing the same point_of_interconnection
direct_matches_expanded = []
for _, row in direct_matches.iterrows():
    qids = row['q_id']
    poi = row['point_of_interconnection']
    # Generate all possible unique pairs
    for i in range(len(qids)):
        for j in range(i+1, len(qids)):
            direct_matches_expanded.append({
                'match_type': 'Direct',
                'point_of_interconnection_1': poi,
                'q_id_1': qids[i],
                'point_of_interconnection_2': poi,
                'q_id_2': qids[j],
                'similarity_score': 100
            })

# Fuzzy matches: already have pairs
fuzzy_matches_expanded = []
for _, row in fuzzy_matches_df.iterrows():
    fuzzy_matches_expanded.append({
        'match_type': 'Fuzzy',
        'point_of_interconnection_1': row['point_of_interconnection_1'],
        'q_id_1': row['q_ids_1'],
        'point_of_interconnection_2': row['point_of_interconnection_2'],
        'q_id_2': row['q_ids_2'],
        'similarity_score': row['similarity_score']
    })

# Convert to DataFrame
matched_qids_df = pd.DataFrame(direct_matches_expanded + fuzzy_matches_expanded)

# Save matched q_ids to CSV
try:
    matched_qids_df.to_csv(MATCHED_QIDS_CSV_PATH, index=False)
    print(f"Matched Q_ids saved to '{MATCHED_QIDS_CSV_PATH}'.")
except Exception as e:
    print(f"Error saving matched Q_ids CSV: {e}")

# ---------------------- Summary ---------------------- #

# Print a summary
total_checked = len(comparison_df)
total_mismatches = len(mismatches_df)
print(f"\nTotal checks performed: {total_checked}")
print(f"Total mismatches found: {total_mismatches}")


Loaded itemized data from /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 7/costs_phase_1_cluster_7_style_G_itemized.csv
Loaded totals data from /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 7/costs_phase_1_cluster_7_style_G_total.csv

All q_ids have all required upgrades.
Mismatch: Q_id 1014, Upgrade 'PTO_IF' - Manual Total: 616.0, Reported Total: 0.0
Mismatch: Q_id 1014, Upgrade 'RNU' - Manual Total: 2035.9, Reported Total: 0.0
Mismatch: Q_id 1027, Upgrade 'RNU' - Manual Total: 1058.1, Reported Total: 0.0

Mismatches saved to '/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 7/mismatches.csv'.

Direct Matches (Exact Point of Interconnection Names):
       point_of_interconnection          q_id
3   230 KV OTAY MESA SWITCHYARD  [1057, 1058]
19         ARCO SUBSTATION 70KV  [1035, 1038]
21       GATES SUBSTATION 230KV  [1027, 1031]
27    MENDOTA SUBSTATION 115 KV  [1028, 1029]

Fuzzy Matches (>=80% Similar

# Checking final data

In [39]:
import pandas as pd
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# ---------------------- Configuration ---------------------- #

# Paths to the CSV files
ITEMIZED_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/01_clean/costs_phase_1_cluster_7_style_G_itemized_updated.csv'
TOTALS_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/01_clean/costs_phase_1_cluster_7_style_G_total_updated.csv'

# Columns in totals_df that hold the reported total costs
TOTALS_ESTIMATED_COLUMN = 'estimated_cost_x_1000'
TOTALS_ESCALATED_COLUMN = 'escalated_cost_x_1000'

# Upgrade types to check
REQUIRED_UPGRADES = ['PTO_IF', 'RNU', 'LDNU', 'ADNU']

# Output paths
MISMATCHES_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/mismatches.csv'
MATCHED_QIDS_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/matched_qids.csv'

# ---------------------- Load Data ---------------------- #

def load_csv(path, dataset_name):
    """
    Loads a CSV file into a pandas DataFrame.
    """
    try:
        df = pd.read_csv(path)
        print(f"Loaded {dataset_name} from {path}")
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {path}")
        exit(1)
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")
        exit(1)

# Load datasets
itemized_df = load_csv(ITEMIZED_CSV_PATH, "itemized data")
totals_df = load_csv(TOTALS_CSV_PATH, "totals data")

# ---------------------- Data Cleaning ---------------------- #

def clean_text(df, column):
    """
    Cleans text data by stripping leading/trailing spaces and converting to uppercase.
    """
    if column in df.columns:
        df[column] = df[column].astype(str).str.strip().str.upper()
    else:
        print(f"Warning: '{column}' column is missing in the dataset. Filling with 'UNKNOWN'.")
        df[column] = 'UNKNOWN'
    return df

# Clean 'type_of_upgrade' and 'point_of_interconnection' in both datasets
itemized_df = clean_text(itemized_df, 'type_of_upgrade')
itemized_df = clean_text(itemized_df, 'point_of_interconnection')

totals_df = clean_text(totals_df, 'type_of_upgrade')
totals_df = clean_text(totals_df, 'point_of_interconnection')

# ---------------------- Data Preparation ---------------------- #

# Ensure necessary columns exist in itemized_df
required_itemized_columns = ['q_id', 'type_of_upgrade', 'point_of_interconnection', 'estimated_cost_x_1000', 'escalated_cost_x_1000']
for col in required_itemized_columns:
    if col not in itemized_df.columns:
        print(f"Warning: '{col}' column is missing in the itemized dataset.")
        if col in ['q_id', 'type_of_upgrade', 'point_of_interconnection']:
            itemized_df[col] = 'UNKNOWN'
        else:
            itemized_df[col] = 0

# Ensure necessary columns exist in totals_df
required_totals_columns = ['q_id', 'type_of_upgrade', 'point_of_interconnection', TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]
for col in required_totals_columns:
    if col not in totals_df.columns:
        print(f"Error: '{col}' column is missing in the totals dataset. Cannot proceed.")
        exit(1)

# Convert cost columns to numeric, coercing errors to NaN and filling with 0
cost_columns_itemized = ['estimated_cost_x_1000', 'escalated_cost_x_1000']
for col in cost_columns_itemized:
    itemized_df[col] = pd.to_numeric(itemized_df[col], errors='coerce').fillna(0)

cost_columns_totals = [TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]
for col in cost_columns_totals:
    totals_df[col] = pd.to_numeric(totals_df[col], errors='coerce').fillna(0)

# ---------------------- Calculate Manual Totals ---------------------- #

# Group itemized data by q_id and type_of_upgrade and calculate sums
itemized_grouped = itemized_df.groupby(['q_id', 'type_of_upgrade']).agg({
    'estimated_cost_x_1000': 'sum',
    'escalated_cost_x_1000': 'sum'
}).reset_index()

# Apply preference: Use estimated_cost_x_1000 if sum > 0, else use escalated_cost_x_1000
itemized_grouped['manual_total'] = itemized_grouped.apply(
    lambda row: row['estimated_cost_x_1000'] if row['estimated_cost_x_1000'] > 0 else row['escalated_cost_x_1000'],
    axis=1
)

# ---------------------- Prepare Totals Data ---------------------- #

# Group totals data by q_id and type_of_upgrade and calculate sums
totals_grouped = totals_df.groupby(['q_id', 'type_of_upgrade']).agg({
    TOTALS_ESTIMATED_COLUMN: 'sum',
    TOTALS_ESCALATED_COLUMN: 'sum'
}).reset_index()

# Apply preference: Use estimated_cost_x_1000 if sum > 0, else use escalated_cost_x_1000
totals_grouped['reported_total'] = totals_grouped.apply(
    lambda row: row[TOTALS_ESTIMATED_COLUMN] if row[TOTALS_ESTIMATED_COLUMN] > 0 else row[TOTALS_ESCALATED_COLUMN],
    axis=1
)

# ---------------------- Merge Data ---------------------- #

# Merge the itemized and totals data on q_id and type_of_upgrade
comparison_df = pd.merge(
    itemized_grouped,
    totals_grouped[['q_id', 'type_of_upgrade', 'reported_total']],
    on=['q_id', 'type_of_upgrade'],
    how='left'
)

# ---------------------- Check for Missing Upgrades ---------------------- #

# Identify q_ids that are missing any of the required upgrades
missing_upgrades_report = []
for q_id in comparison_df['q_id'].unique():
    upgrades_present = comparison_df[comparison_df['q_id'] == q_id]['type_of_upgrade'].unique()
    missing_upgrades = [upgrade for upgrade in REQUIRED_UPGRADES if upgrade not in upgrades_present]
    if missing_upgrades:
        missing_upgrades_report.append((q_id, missing_upgrades))

# Report missing upgrades
if missing_upgrades_report:
    print("\nQ_ids with missing upgrades:")
    for q_id, missing in missing_upgrades_report:
        print(f"  Q_id {q_id} is missing upgrades: {', '.join(missing)}")
else:
    print("\nAll q_ids have all required upgrades.")

# ---------------------- Compare Totals and Identify Mismatches ---------------------- #

# Initialize list to store mismatches
mismatches = []

# Iterate through each row to compare manual_total with reported_total
for index, row in comparison_df.iterrows():
    q_id = row['q_id']
    upgrade = row['type_of_upgrade']
    manual_total = row['manual_total']
    reported_total = row['reported_total']
    
    # Determine if both manual_total and reported_total are zero
    if manual_total == 0.0 and reported_total == 0.0:
        continue  # No mismatch
    # Determine if manual_total is zero and reported_total is missing or zero
    elif manual_total == 0.0 and (pd.isna(row['reported_total']) or reported_total == 0.0):
        continue  # No mismatch
    # If reported_total is missing (NaN) and manual_total is not zero
    elif pd.isna(row['reported_total']) and manual_total != 0.0:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: Missing")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': 'Missing'
        })
    # If manual_total is not zero and reported_total is zero
    elif manual_total != 0.0 and reported_total == 0.0:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: 0.0")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': reported_total
        })
    # If both totals are non-zero but differ beyond tolerance
    elif abs(manual_total - reported_total) > 1e-2:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: {reported_total}")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': reported_total
        })
    # Else, totals match; do nothing

# Create a DataFrame for mismatches
mismatches_df = pd.DataFrame(mismatches, columns=['q_id', 'type_of_upgrade', 'manual_total', 'reported_total'])

# Save mismatches to a CSV file
try:
    mismatches_df.to_csv(MISMATCHES_CSV_PATH, index=False)
    print(f"\nMismatches saved to '{MISMATCHES_CSV_PATH}'.")
except Exception as e:
    print(f"Error saving mismatches CSV: {e}")

# ---------------------- Point of Interconnection Matching ---------------------- #

# Extract unique q_id and point_of_interconnection from itemized dataset
itemized_poi = itemized_df[['q_id', 'point_of_interconnection']].drop_duplicates()

# Extract unique q_id and point_of_interconnection from totals dataset
totals_poi = totals_df[['q_id', 'point_of_interconnection']].drop_duplicates()

# Merge both to have a complete list of q_id and point_of_interconnection
all_poi = pd.concat([itemized_poi, totals_poi]).drop_duplicates().reset_index(drop=True)

# ---------------------- Direct Match Identification ---------------------- #

# Group by point_of_interconnection to find q_ids sharing the same point_of_interconnection
direct_matches = all_poi.groupby('point_of_interconnection')['q_id'].apply(list).reset_index()

# Filter groups with more than one q_id (i.e., shared points_of_interconnection)
direct_matches = direct_matches[direct_matches['q_id'].apply(len) > 1]

print("\nDirect Matches (Exact Point of Interconnection Names):")
if not direct_matches.empty:
    print(direct_matches)
else:
    print("No direct matches found.")

# ---------------------- Fuzzy Match Identification ---------------------- #

# Prepare list of points_of_interconnection for fuzzy matching
poi_list = all_poi['point_of_interconnection'].unique().tolist()

# Initialize list to store fuzzy matches
fuzzy_matches = []

# Iterate through each point_of_interconnection to find similar ones
for i, poi in enumerate(poi_list):
    # Compare with the rest of the points to avoid redundant comparisons
    similar_pois = process.extract(poi, poi_list[i+1:], scorer=fuzz.token_set_ratio)
    
    # Filter matches with similarity >= 80%
    for match_poi, score in similar_pois:
        if score >= 80:
            # Retrieve q_ids for both points_of_interconnection
            qids_poi1 = all_poi[all_poi['point_of_interconnection'] == poi]['q_id'].tolist()
            qids_poi2 = all_poi[all_poi['point_of_interconnection'] == match_poi]['q_id'].tolist()
            
            # Append the matched pairs with their points_of_interconnection and similarity score
            fuzzy_matches.append({
                'point_of_interconnection_1': poi,
                'q_ids_1': qids_poi1,
                'point_of_interconnection_2': match_poi,
                'q_ids_2': qids_poi2,
                'similarity_score': score
            })

# Convert fuzzy matches to DataFrame
fuzzy_matches_df = pd.DataFrame(fuzzy_matches)

print("\nFuzzy Matches (>=80% Similarity in Point of Interconnection):")
if not fuzzy_matches_df.empty:
    print(fuzzy_matches_df)
else:
    print("No fuzzy matches found.")

# ---------------------- Save Matched Q_ids to CSV ---------------------- #

# For clarity, create a combined DataFrame for direct and fuzzy matches

# Direct matches: list each pair of q_ids sharing the same point_of_interconnection
direct_matches_expanded = []
for _, row in direct_matches.iterrows():
    qids = row['q_id']
    poi = row['point_of_interconnection']
    # Generate all possible unique pairs
    for i in range(len(qids)):
        for j in range(i+1, len(qids)):
            direct_matches_expanded.append({
                'match_type': 'Direct',
                'point_of_interconnection_1': poi,
                'q_id_1': qids[i],
                'point_of_interconnection_2': poi,
                'q_id_2': qids[j],
                'similarity_score': 100
            })

# Fuzzy matches: already have pairs
fuzzy_matches_expanded = []
for _, row in fuzzy_matches_df.iterrows():
    fuzzy_matches_expanded.append({
        'match_type': 'Fuzzy',
        'point_of_interconnection_1': row['point_of_interconnection_1'],
        'q_id_1': row['q_ids_1'],
        'point_of_interconnection_2': row['point_of_interconnection_2'],
        'q_id_2': row['q_ids_2'],
        'similarity_score': row['similarity_score']
    })

# Convert to DataFrame
matched_qids_df = pd.DataFrame(direct_matches_expanded + fuzzy_matches_expanded)

# Save matched q_ids to CSV
try:
    matched_qids_df.to_csv(MATCHED_QIDS_CSV_PATH, index=False)
    print(f"Matched Q_ids saved to '{MATCHED_QIDS_CSV_PATH}'.")
except Exception as e:
    print(f"Error saving matched Q_ids CSV: {e}")

# ---------------------- Summary ---------------------- #

# Print a summary
total_checked = len(comparison_df)
total_mismatches = len(mismatches_df)
print(f"\nTotal checks performed: {total_checked}")
print(f"Total mismatches found: {total_mismatches}")


Loaded itemized data from /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 7/01_clean/costs_phase_1_cluster_7_style_G_itemized_updated.csv
Loaded totals data from /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 7/01_clean/costs_phase_1_cluster_7_style_G_total_updated.csv

All q_ids have all required upgrades.
Mismatch: Q_id 1014, Upgrade 'PTO_IF' - Manual Total: 616.0, Reported Total: 0.0
Mismatch: Q_id 1014, Upgrade 'RNU' - Manual Total: 2035.9, Reported Total: 0.0
Mismatch: Q_id 1027, Upgrade 'RNU' - Manual Total: 1058.1, Reported Total: 0.0

Mismatches saved to '/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 7/mismatches.csv'.

Direct Matches (Exact Point of Interconnection Names):
       point_of_interconnection          q_id
3   230 KV OTAY MESA SWITCHYARD  [1057, 1058]
19         ARCO SUBSTATION 70KV  [1035, 1038]
21       GATES SUBSTATION 230KV  [1027, 1031]
27    MENDOTA SUBSTATION 115 KV  [1028, 1